In [1]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# PERIODO

In [2]:
start_date = '2024-01-01'
final_date = '2024-01-31'

final_date_1 = '2024-01-10'
start_date_1 = '2024-01-11'

final_date_2 = '2024-01-20'
start_date_2 = '2024-01-21'

final_date_3 = '2024-01-25'
start_date_3 = '2024-01-26'

# BIN AMEX

In [3]:
bin_amex = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\BIN AMEX.txt',dtype={'BIN_RANGE':object,'COUNTRY_CODE':object})
bin_amex.fillna('null',inplace=True)
bin_amex.loc[bin_amex['COUNTRY_CODE'].str.contains('484'), "origen_tarjeta"] = 'NACIONAL'
bin_amex['origen_tarjeta'].fillna('INTERNACIONAL',inplace=True)
bin_amex['bin_bank'] = (bin_amex['BIN_RANGE'].astype(str).replace(' ', '',regex=True))
bin_amex.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\4226851102.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'null' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bin_amex.fillna('null',inplace=True)


,BIN_RANGE,PRODUCT_TYPE,COUNTRY_CODE,COUNTRY_NAME,CPS_OPTION,20211212,origen_tarjeta,bin_bank
0,370395,DEBIT,840,UNITED STATES OF AMERICA,N,null,nan,370395
1,379983,CONSUMER,156,CHINA,N,null,nan,379983
2,378256,CONSUMER,156,CHINA,N,null,nan,378256
3,370524,CONSUMER,156,CHINA,N,null,nan,370524
4,377659,CONSUMER,156,CHINA,N,null,nan,377659


In [4]:
bin_amex.shape

(17501, 8)

In [5]:
bin_amex.to_csv('bines.csv')

# PARAMETRIA COMISIONES

In [6]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\PARAMETRIA AMEX.csv',dtype={'meses_msi':object,
                                                                                              'comision_porcentaje':float,
                                                                                              'industria_afiliacion':object})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
#parametria_comisiones['fin'] = pd.to_datetime(parametria_comisiones['fin'])
parametria_comisiones['comision_porcentaje_mdr'] = -0.0240
parametria_comisiones['comision_porcentaje_msi'] = -parametria_comisiones['comision_porcentaje']/100
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('EPA'), "affiliation_type"] = 'AGGREGATOR'
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('GRRCN'), "affiliation_type"] = 'OPTBLUE'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('99'),'meses_msi'] = '0'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('0'),'comision_porcentaje_msi'] = 0
parametria_comisiones['comision_porcentaje_final'] = (parametria_comisiones['comision_porcentaje_mdr'] +
                                                      parametria_comisiones['comision_porcentaje_msi'])
parametria_comisiones['index_comisiones'] = (parametria_comisiones['affiliation_type'].astype(str).replace(' ', '',regex=True)+'_'+
                                             parametria_comisiones['meses_msi'].astype(str).replace(' ', '',regex=True))
parametria_comisiones['index_comisiones_industria'] = (parametria_comisiones['industria_afiliacion'].astype(str).replace(' ', '',regex=True)+'_'+
                                                    (parametria_comisiones['affiliation_type'].astype(str).replace(' ', '',regex=True))+'_'+
                                                    (parametria_comisiones['meses_msi'].astype(str).replace(' ', '',regex=True)))
parametria_comisiones.loc[~(parametria_comisiones['index_comisiones_industria'].str.contains('TODAS')), "index_comisiones"] = 'NO_CONSIDERAR'
parametria_comisiones.head()

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,proceso,comentario,claves_comision,status_clip,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones,index_comisiones_industria
0,Null,Fija,VIGENTE,2018-01-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,EPA,...,Agregador,Costo de txn Amex Agregador General,NaN,1,-0.02,0.00,AGGREGATOR,-0.02,NO_CONSIDERAR,9351789434_AGGREGATOR_0
1,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.03,AGGREGATOR,-0.05,NO_CONSIDERAR,9351789434_AGGREGATOR_3
2,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,Agregador,Costo por el uso de MSI adicional a la tasa y ...,NaN,1,-0.02,-0.00,AGGREGATOR,-0.03,NO_CONSIDERAR,9351789434_AGGREGATOR_3
3,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.04,AGGREGATOR,-0.07,NO_CONSIDERAR,9351789434_AGGREGATOR_6
4,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,Agregador,Costo por el uso de MSI adicional a la tasa y ...,NaN,1,-0.02,-0.00,AGGREGATOR,-0.03,NO_CONSIDERAR,9351789434_AGGREGATOR_6


In [7]:
parametria_comisiones.shape

(55, 30)

In [8]:
parametria_comisiones['fuente'].unique()

array(['EPA', 'GRRCN'], dtype=object)

In [9]:
parametria_comisiones['meses_msi'].unique()

array(['0', '3', '6', '9', '12', '18', '15'], dtype=object)

# COMISIONES GRRCN COBRADAS

In [10]:
comisiones_grrcn = parametria_comisiones[(parametria_comisiones['fuente'] == 'GRRCN') &
                                       (parametria_comisiones['status'].str.contains('VIGENTE')) &
                                       (~parametria_comisiones['fin'].str.contains('2021-04-15'))]
comisiones_grrcn.head()

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,proceso,comentario,claves_comision,status_clip,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones,index_comisiones_industria
15,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,OptBlue,NF Network Fee Uso de Red,NF,1,-0.02,0.00,OPTBLUE,-0.02,OPTBLUE_0,TODAS_OPTBLUE_0
16,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,OptBlue,IB Inbound fee Uso de tarjeta internacional,IB-IP-IS,1,-0.02,0.00,OPTBLUE,-0.02,OPTBLUE_0,TODAS_OPTBLUE_0
17,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,OptBlue,1A MDR Costo de tasa industria EDUCATION,1A-7G,1,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464193_OPTBLUE_0
18,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,OptBlue,1A MDR Costo de tasa industria AUTO RENTAL,1A-7G,1,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464219_OPTBLUE_0
19,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,OptBlue,1A MDR Costo de tasa industria RETAIL,1A-7G,1,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464227_OPTBLUE_0


In [11]:
comisiones_grrcn.shape

(34, 30)

In [12]:
comisiones_grrcn['index_comisiones'].unique()

array(['OPTBLUE_0', 'NO_CONSIDERAR', 'OPTBLUE_3', 'OPTBLUE_6',
       'OPTBLUE_9', 'OPTBLUE_12', 'OPTBLUE_15', 'OPTBLUE_18'],
      dtype=object)

In [13]:
searchfor_card_present = ['8138464193',
'8138464219',
'8138464227',
'8138464185',
'8138464177',
'8138464151',
'8138464144',
'8138464136',
'8138464128',
'8138464441',
'8138464110']
searchfor_card_not_present = ['8482702172',
'8482702149',
'8482707999',
'8482702206',
'8482702263',
'8482702271',
'8482702297',
'8482707973',
'8482708005',
'8482702248',
'8482708013',
'8482702164']

In [14]:
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_present)), "industria_fee"] = comisiones_grrcn['comision_porcentaje']/100
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_not_present)), "industria_fee"] = comisiones_grrcn['comision_porcentaje']/100
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_present)), "tipo_tarjeta"] = 'card_present'
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_not_present)), "tipo_tarjeta"] = 'card_not_present'
comisiones_grrcn.loc[(comisiones_grrcn['fuente'] == 'GRRCN'), "network_fee"] = 0.2/100
comisiones_grrcn.loc[~(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')), "sobretasa"] = comisiones_grrcn['comision_porcentaje']/100
comisiones_grrcn.loc[~(comisiones_grrcn['comision_aplicada'].str.contains('SOBRE TASA')), "index_comisiones"] = 'NO_CONSIDERAR'
comisiones_grrcn['index_comisiones_industria'] = comisiones_grrcn['index_comisiones_industria'].str[:-2]
comisiones_grrcn['industria_fee'].fillna(0,inplace=True)
comisiones_grrcn['sobretasa'].fillna(0,inplace=True)
comisiones_grrcn.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3320542273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3320542273.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3320

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones,index_comisiones_industria,industria_fee,tipo_tarjeta,network_fee,sobretasa
15,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,TODAS_OPTBLUE,0.00,nan,0.00,0.00
16,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,TODAS_OPTBLUE,0.00,nan,0.00,0.00
17,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464193_OPTBLUE,0.01,card_present,0.00,0.00
18,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464219_OPTBLUE,0.02,card_present,0.00,0.00
19,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464227_OPTBLUE,0.01,card_present,0.00,0.00


In [15]:
comisiones_grrcn.shape

(34, 34)

In [16]:
comisiones_grrcn['industria_fee'].unique()

array([0.    , 0.0065, 0.0165, 0.015 , 0.013 , 0.018 , 0.0105, 0.0185,
       0.0195, 0.004 , 0.016 ])

In [17]:
comisiones_grrcn['index_comisiones'].unique()

array(['NO_CONSIDERAR', 'OPTBLUE_3', 'OPTBLUE_6', 'OPTBLUE_9',
       'OPTBLUE_12', 'OPTBLUE_15', 'OPTBLUE_18'], dtype=object)

In [18]:
comisiones_grrcn.to_csv('comisiones_grrcn.csv')

# DB

In [19]:
# db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX SEPTIEMBRE 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
# db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX AGOSTO 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
# db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX AGOSTO-SEPTIEMBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX OCTUBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_3 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX JUNIO-JULIO 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_4 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX AGOSTO-SEPTIEMBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2024\AMEX ENERO 2024.csv'
                   ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
                            ,'MCC':object})

In [20]:
db_all = pd.concat([db_1])
db_all.columns = db_all.columns.str.strip().str.lower() 
db_all['order_id'] = db_all['order_id'].str.strip()
db_all['order_id'] = db_all['order_id'].str.upper()
db_all['date_transaction'] = pd.to_datetime(db_all['date_transaction'])
db_all['INDEX_TOTAL'] = (db_all['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_all['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_all['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_all['amount'].astype(str).replace(' ', '',regex=True))
db_all['INDEX_PARCIAL'] = (db_all['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_all['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_all['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_all['amount'].astype(str).replace(' ', '',regex=True))
db_all['order_id'].fillna('SIN ORDER_ID EN DB',inplace=True)
db_all.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,status_code,decline_code,msi,device_id,clip_reader,status_msg,issuer,brand,INDEX_TOTAL,INDEX_PARCIAL
0,c090de31-388f-41c9-9bf0-5ae7adf17975,5725dcb9-4054-4cb0-94ff-a9be5b970b4b,S0CKBKMHDQPP,010339200576064,8138464136,OPTBLUE,710910,5812,2023-12-25,2023-12-25T06:05:50Z,...,1,NaN,0.00000,P5220314003479,Clip Classic,Paid,AMEX,American Express,379794_1004_S0CKBKMHDQPP_380.0,379794_1004_010339200576064_380.0
1,4a9c6399-ae89-4b68-9607-cbd05f43e110,dcac4d6b-cd68-40c6-af7f-1999753fe76c,Z5MDOPDNOIPD,010339208448068,8138464136,OPTBLUE,1908592,5813,2023-12-25,2023-12-25T06:06:06Z,...,1,NaN,0.00000,N500W4P2184,Clip Total,Paid,AMEX,American Express,371292_3000_Z5MDOPDNOIPD_3210.0,371292_3000_010339208448068_3210.0
2,8c22ed4b-9555-4018-8fcb-c05df672c953,54d07a23-3169-44ac-b3a1-44d6b781fe70,HZJNZXXXDMRT,010339220050066,8138464136,OPTBLUE,1658106,5813,2023-12-25,2023-12-25T06:06:58Z,...,1,NaN,0.00000,N500W4M8642,Clip Total,Paid,AMEX,AMERICAN EXPRESS,376667_2037_HZJNZXXXDMRT_200.0,376667_2037_010339220050066_200.0
3,3b5049b8-3217-49ca-86ea-caaaa3ec3308,c90ef4ae-3349-4a07-af68-6fb916b8e67a,5IT5LIBALHVF,010339219163060,8138464136,OPTBLUE,1106905,5813,2023-12-25,2023-12-25T06:07:45Z,...,1,NaN,0.00000,N500W4E2326,Clip Total,Paid,AMEX,American Express,377978_1006_5IT5LIBALHVF_180.0,377978_1006_010339219163060_180.0
4,8c22ed4b-9555-4018-8fcb-c05df672c953,548bd5a9-596a-4803-86c2-6e0d95f7e2cb,MMX2JP07EXV1,010339204880066,8138464136,OPTBLUE,1658106,5813,2023-12-25,2023-12-25T06:08:21Z,...,1,NaN,0.00000,N500W4M8642,Clip Total,Paid,AMEX,AMERICAN EXPRESS,376718_1042_MMX2JP07EXV1_200.0,376718_1042_010339204880066_200.0


In [21]:
db_all.shape

(337597, 26)

In [22]:
duplicates_db = db_all[db_all.duplicated(['transaction_id'] , keep=False)]
duplicates_db.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,status_code,decline_code,msi,device_id,clip_reader,status_msg,issuer,brand,INDEX_TOTAL,INDEX_PARCIAL
321803,62ab9ad3-50e4-4c56-b7fe-4b0d6691b426,40f02f7b-abd8-4a35-bb8a-7dfdf4490405,OC2WO7ERVMNY,NaN,8138464144,OPTBLUE,2015017,7399,2024-01-30,2024-01-30T17:08:54Z,...,-3,NaN,0.00000,80010001,Clip Classic,Deny,AMEX,AMERICAN EXPRESS,371776_2007_OC2WO7ERVMNY_17214.33,371776_2007_nan_17214.33
321804,62ab9ad3-50e4-4c56-b7fe-4b0d6691b426,40f02f7b-abd8-4a35-bb8a-7dfdf4490405,OC2WO7ERVMNY,NaN,8138464144,ECOMMERCE,2015017,7399,2024-01-30,2024-01-30T17:08:54Z,...,-3,NaN,0.00000,80010001,Clip Classic,Deny,AMEX,AMERICAN EXPRESS,371776_2007_OC2WO7ERVMNY_17214.33,371776_2007_nan_17214.33
321864,ceb5c5ea-1735-4745-9389-43122cf0b98e,5d6be542-93bf-4498-a05a-9c39f1e4cec0,OLUXGABGTF4U,010901004781065,8138464227,ECOMMERCE,9686,5499,2024-01-30,2024-01-30T17:14:53Z,...,1,NaN,0.00000,80010001,Clip Classic,Paid,AMEXBANK,AMERICAN EXPRESS,371690_7005_OLUXGABGTF4U_36103.89,371690_7005_010901004781065_36103.89
321865,ceb5c5ea-1735-4745-9389-43122cf0b98e,5d6be542-93bf-4498-a05a-9c39f1e4cec0,OLUXGABGTF4U,010901004781065,8138464227,OPTBLUE,9686,5499,2024-01-30,2024-01-30T17:14:53Z,...,1,NaN,0.00000,80010001,Clip Classic,Paid,AMEXBANK,AMERICAN EXPRESS,371690_7005_OLUXGABGTF4U_36103.89,371690_7005_010901004781065_36103.89


In [23]:
duplicates_db.shape

(4, 26)

In [24]:
db_all['affiliation_type'].unique()

array(['OPTBLUE', nan, 'AGGREGATOR', 'CARD_PRESENT', 'ECOMMERCE'],
      dtype=object)

### GRRCN ONLY

In [25]:
db = db_all

In [26]:
# db = db_all[(db_all['affiliation_type'] == ('OPTBLUE'))]
# db.head()

In [27]:
db.shape

(337597, 26)

# GRRNC

In [28]:
allfiles = glob.glob(r'G:\.shortcut-targets-by-id\1dIw3KdQWk6HAGd7cIi1VZsIyka0tkfYi\2.-CONCILIACIONES Y POLIZA\GRRCN\2024\**\OPTBLUE - PRODUCCION\*.txt', recursive=True)
allfiles

['G:\\.shortcut-targets-by-id\\1dIw3KdQWk6HAGd7cIi1VZsIyka0tkfYi\\2.-CONCILIACIONES Y POLIZA\\GRRCN\\2024\\01. ENERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2024-01-01.txt',
 'G:\\.shortcut-targets-by-id\\1dIw3KdQWk6HAGd7cIi1VZsIyka0tkfYi\\2.-CONCILIACIONES Y POLIZA\\GRRCN\\2024\\01. ENERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2024-01-03.txt',
 'G:\\.shortcut-targets-by-id\\1dIw3KdQWk6HAGd7cIi1VZsIyka0tkfYi\\2.-CONCILIACIONES Y POLIZA\\GRRCN\\2024\\01. ENERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2024-01-02.txt',
 'G:\\.shortcut-targets-by-id\\1dIw3KdQWk6HAGd7cIi1VZsIyka0tkfYi\\2.-CONCILIACIONES Y POLIZA\\GRRCN\\2024\\01. ENERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2024-01-04.txt',
 'G:\\.shortcut-targets-by-id\\1dIw3KdQWk6HAGd7cIi1VZsIyka0tkfYi\\2.-CONCILIACIONES Y POLIZA\\GRRCN\\2024\\01. ENERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2024-01-05.txt',
 'G:\\.shortcut-targets-by-id\\1dIw3KdQWk6HAGd7cIi1VZsIyka0tkfYi\\2.-CONCILIACIONES Y POLIZA\\GRRCN\\2024\\01. ENERO\\OPTBLUE - PRODUCCION\\OPTBL

In [29]:
names_header=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'
              ,'22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']

In [30]:
grrcn_file = pd.concat([pd.read_csv(f,names=names_header, sep=',',dtype=object) for f in allfiles])
grrcn_file.head()

,1,2,3,4,5,6,7,8,9,10,...,32,33,34,35,36,37,38,39,40,41
0,HEADER,20240101,031522,0000001056,GRRCN,PAYCLIP MEXICO,3.01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAILER,0000001056,0000000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,HEADER,20240103,010121,0000001058,GRRCN,PAYCLIP MEXICO,3.01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SUMMARY,7464284207,002,NaN,20240103,MXN,NaN,-000000001050179,000000000000000,000000000047779,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SUBMISSION,7464284207,002,NaN,20240103,MXN,8138464151,20240102,20240102,813846415140102,...,000000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
grrcn_file.shape

(3142454, 41)

In [32]:
grrcn_file['1'].unique()

array(['HEADER', 'TRAILER', 'SUMMARY', 'SUBMISSION', 'TRANSACTN',
       'TXNPRICING', 'ADJUSTMENT', 'CHARGEBACK'], dtype=object)

In [33]:
grrcn_head = grrcn_file[(grrcn_file['1'] == 'HEADER')]
grrcn_summ = grrcn_file[(grrcn_file['1'] == 'SUMMARY')]
grrcn_subm = grrcn_file[(grrcn_file['1'] == 'SUBMISSION')]
grrcn_tran = grrcn_file[(grrcn_file['1'] == 'TRANSACTN')]
grrcn_txnp = grrcn_file[(grrcn_file['1'] == 'TXNPRICING')]
grrcn_char = grrcn_file[(grrcn_file['1'] == 'CHARGEBACK')]
grrcn_adju = grrcn_file[(grrcn_file['1'] == 'ADJUSTMENT')]
grrcn_trai = grrcn_file[(grrcn_file['1'] == 'TRAILER')]

### SUMMARY

In [34]:
grrcn_summ['5'] = pd.to_datetime(grrcn_summ['5'], format='%Y%m%d').dt.date
grrcn_summ['5'] = pd.to_datetime(grrcn_summ['5'])
grrcn_summ.sort_values(by=['5'],inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3960028797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_summ['5'] = pd.to_datetime(grrcn_summ['5'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3960028797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_summ['5'] = pd.to_datetime(grrcn_summ['5'])
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3960028797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a 

In [35]:
grrcn_summ.rename(columns={'5':'fecha_pago'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\723792245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_summ.rename(columns={'5':'fecha_pago'},inplace=True)


In [36]:
grrcn_summ['8'] = grrcn_summ['8'].astype(float)
grrcn_summ['9'] = grrcn_summ['9'].astype(float)
grrcn_summ['10'] = grrcn_summ['10'].astype(float)
grrcn_summ['11'] = grrcn_summ['11'].astype(float)
grrcn_summ['12'] = grrcn_summ['12'].astype(float)
grrcn_summ['13'] = grrcn_summ['13'].astype(float)
grrcn_summ['14'] = grrcn_summ['14'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\429899166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_summ['8'] = grrcn_summ['8'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\429899166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_summ['9'] = grrcn_summ['9'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\429899166.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [37]:
grrcn_summ['Monto_neto'] = grrcn_summ['8']/100
grrcn_summ['Monto_bruto'] = grrcn_summ['9']/100
grrcn_summ['Monto_descuento'] = grrcn_summ['10']/100
grrcn_summ['Monto_servicio'] = grrcn_summ['11']/100
grrcn_summ['Monto_ajuste'] = grrcn_summ['12']/100
grrcn_summ['Monto_impuestos'] = grrcn_summ['13']/100
grrcn_summ['Monto_saldo_debito_inicial'] = grrcn_summ['14']/100
grrcn_summ.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2700658038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_summ['Monto_neto'] = grrcn_summ['8']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2700658038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_summ['Monto_bruto'] = grrcn_summ['9']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2700658038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,1,2,3,4,fecha_pago,6,7,8,9,10,...,39,40,41,Monto_neto,Monto_bruto,Monto_descuento,Monto_servicio,Monto_ajuste,Monto_impuestos,Monto_saldo_debito_inicial
1,SUMMARY,7464284207,002,NaN,2024-01-02,MXN,NaN,"-995,203.00",0.00,"238,780.00",...,NaN,NaN,NaN,"-9,952.03",0.00,"2,387.80",24.00,0.00,385.88,"-7,154.35"
754,SUMMARY,8138352885,002,3630036365,2024-01-02,MXN,NaN,"7,362,518,997.00","7,610,454,795.00","156,575,445.00",...,NaN,NaN,NaN,"73,625,189.97","76,104,547.95","1,565,754.45","7,984.80","-653,820.70","251,798.03",0.00
1,SUMMARY,7464284207,002,NaN,2024-01-03,MXN,NaN,"-1,050,179.00",0.00,"47,779.00",...,NaN,NaN,NaN,"-10,501.79",0.00,477.79,8.50,14.33,77.80,"-9,952.03"
189,SUMMARY,8138352885,002,0020024044,2024-01-03,MXN,NaN,"1,644,093,684.00","1,708,025,318.00","34,084,092.00",...,NaN,NaN,NaN,"16,440,936.84","17,080,253.18","340,840.92","1,642.50","-242,035.69","54,797.23",0.00
1,SUMMARY,7464284207,002,NaN,2024-01-04,MXN,NaN,"-1,114,130.00",0.00,"54,390.00",...,NaN,NaN,NaN,"-11,141.30",0.00,543.90,7.40,0.00,88.21,"-10,501.79"


In [38]:
grrcn_summ.shape

(50, 48)

In [39]:
grrcn_summ_range = grrcn_summ[(grrcn_summ['fecha_pago'] >= start_date) & (grrcn_summ['fecha_pago'] <= final_date)]

### SUBMISSION

In [40]:
grrcn_subm['5'] = pd.to_datetime(grrcn_subm['5'], format='%Y%m%d').dt.date
grrcn_subm['8'] = pd.to_datetime(grrcn_subm['8'], format='%Y%m%d').dt.date
grrcn_subm['9'] = pd.to_datetime(grrcn_subm['9'], format='%Y%m%d').dt.date
grrcn_subm['26'] = pd.to_datetime(grrcn_subm['26'], format='%Y%m%d').dt.date
grrcn_subm['5'] = pd.to_datetime(grrcn_subm['5'])
grrcn_subm['8'] = pd.to_datetime(grrcn_subm['8'])
grrcn_subm['9'] = pd.to_datetime(grrcn_subm['9'])
grrcn_subm.sort_values(by=['5'],inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\4035782578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_subm['5'] = pd.to_datetime(grrcn_subm['5'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\4035782578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_subm['8'] = pd.to_datetime(grrcn_subm['8'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\4035782578.py:3: SettingWithCopyWarning: 
A value 

In [41]:
grrcn_subm.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_envio_comercial','9':'fecha_procesamiento'
                          ,'26':'fecha_pago_original'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2539011656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_subm.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_envio_comercial','9':'fecha_procesamiento'


In [42]:
grrcn_subm['14'] = grrcn_subm['14'].astype(float)
grrcn_subm['15'] = grrcn_subm['15'].astype(float)
grrcn_subm['16'] = grrcn_subm['16'].astype(float)
grrcn_subm['17'] = grrcn_subm['17'].astype(float)
grrcn_subm['18'] = grrcn_subm['18'].astype(float)
grrcn_subm['19'] = grrcn_subm['19'].astype(float)
grrcn_subm['29'] = grrcn_subm['29'].astype(float)
grrcn_subm['30'] = grrcn_subm['30'].astype(float)
grrcn_subm['31'] = grrcn_subm['31'].astype(float)
grrcn_subm['32'] = grrcn_subm['32'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3717189171.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_subm['14'] = grrcn_subm['14'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3717189171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_subm['15'] = grrcn_subm['15'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3717189171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [43]:
grrcn_subm['Monto_bruto'] = grrcn_subm['14']/100
grrcn_subm['Monto_bruto_moneda_pago'] = grrcn_subm['15']/100
grrcn_subm['Monto_descuento'] = grrcn_subm['16']/100
grrcn_subm['Monto_servicio'] = grrcn_subm['17']/100
grrcn_subm['Monto_impuestos'] = grrcn_subm['18']/100
grrcn_subm['Monto_neto'] = grrcn_subm['19']/100
grrcn_subm['Monto_cargo_aceleracion'] = grrcn_subm['29']/100
grrcn_subm['Monto_neto_cargo_aceleracion'] = grrcn_subm['30']/100
grrcn_subm['Monto_bruto_debito'] = grrcn_subm['31']/100
grrcn_subm['Monto_bruto_credito'] = grrcn_subm['32']/100
grrcn_subm.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2011396041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_subm['Monto_bruto'] = grrcn_subm['14']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2011396041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_subm['Monto_bruto_moneda_pago'] = grrcn_subm['15']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2011396041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_envio_comercial,fecha_procesamiento,10,...,Monto_bruto,Monto_bruto_moneda_pago,Monto_descuento,Monto_servicio,Monto_impuestos,Monto_neto,Monto_cargo_aceleracion,Monto_neto_cargo_aceleracion,Monto_bruto_debito,Monto_bruto_credito
179799,SUBMISSION,8138352885,002,3630036365,2024-01-02,MXN,8138464227,2023-12-29,2023-12-29,813846422731230,...,"1,719,721.92","1,719,721.92","31,737.85",187.00,"5,107.96","1,682,689.11",0.00,0.00,"1,719,721.92",0.00
91763,SUBMISSION,8138352885,002,3630036365,2024-01-02,MXN,8138464193,2023-12-29,2023-12-31,813846419340101,...,"62,480.00","62,480.00",550.80,2.80,88.58,"61,837.82",0.00,0.00,"62,480.00",0.00
91803,SUBMISSION,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,2023-12-29,813846413631230,...,"1,659,838.76","1,659,838.76","38,108.35",211.10,"6,131.11","1,615,388.20",0.00,0.00,"1,659,838.76",0.00
99145,SUBMISSION,8138352885,002,3630036365,2024-01-02,MXN,8138464193,2023-12-29,2023-12-31,813846419340101,...,"1,685.38","1,685.38",14.32,0.40,2.35,"1,668.31",0.00,0.00,"1,685.38",0.00
99153,SUBMISSION,8138352885,002,3630036365,2024-01-02,MXN,8138464193,2023-12-29,2023-12-31,813846419340101,...,"2,500.00","2,500.00",21.25,0.40,3.46,"2,474.89",0.00,0.00,"2,500.00",0.00


In [44]:
grrcn_subm.shape

(3359, 51)

In [45]:
grrcn_subm_range = grrcn_subm[(grrcn_subm['fecha_pago'] >= start_date) & (grrcn_subm['fecha_pago'] <= final_date)]

### TRANSACTN 

In [46]:
grrcn_tran['5'] = pd.to_datetime(grrcn_tran['5'], format='%Y%m%d').dt.date
grrcn_tran['8'] = pd.to_datetime(grrcn_tran['8'], format='%Y%m%d').dt.date
grrcn_tran['9'] = pd.to_datetime(grrcn_tran['9'], format='%Y%m%d').dt.date
grrcn_tran['19'] = pd.to_datetime(grrcn_tran['19'], format='%Y%m%d').dt.date
grrcn_tran['20'] = pd.to_datetime(grrcn_tran['20'], format='%H%M%S').dt.time
grrcn_tran['5'] = pd.to_datetime(grrcn_tran['5'])
grrcn_tran['8'] = pd.to_datetime(grrcn_tran['8'])
grrcn_tran['9'] = pd.to_datetime(grrcn_tran['9'])
grrcn_tran['19'] = pd.to_datetime(grrcn_tran['19'])
grrcn_tran.sort_values(by=['19','20'],inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3656770900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['5'] = pd.to_datetime(grrcn_tran['5'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3656770900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['8'] = pd.to_datetime(grrcn_tran['8'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3656770900.py:3: SettingWithCopyWarning: 
A value 

In [47]:
grrcn_tran.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_presentacion','9':'fecha_procesamiento'
                           ,'15':'no_tarjeta','13':'order_id','22':'codigo_aprobacion','23':'numero_terminal'
                           ,'26':'referencia_adquiriente','36':'cantidad_cuotas','37':'numero_cuota'
                          ,'19':'fecha_transaccion','20':'hora_transaccion','21':'numero_transaccion',
                          '33':'rechazo_amex'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2639368825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_presentacion','9':'fecha_procesamiento'


In [48]:
grrcn_tran['17'] = grrcn_tran['17'].astype(float)
grrcn_tran['18'] = grrcn_tran['18'].astype(float)
grrcn_tran['34'] = grrcn_tran['34'].astype(float)
grrcn_tran['35'] = grrcn_tran['35'].astype(float)
grrcn_tran['39'] = grrcn_tran['39'].astype(float)
grrcn_tran['40'] = grrcn_tran['40'].astype(float)
grrcn_tran['cantidad_cuotas'] = grrcn_tran['cantidad_cuotas'].astype(float)
grrcn_tran['numero_cuota'] = grrcn_tran['numero_cuota'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2629539950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['17'] = grrcn_tran['17'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2629539950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['18'] = grrcn_tran['18'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2629539950.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [49]:
grrcn_tran['Monto_bruto_presentacion'] = grrcn_tran['17']/100
grrcn_tran['Monto_transaccion'] = grrcn_tran['18']/100
grrcn_tran['Monto_primera_cuota'] = grrcn_tran['34']/100
grrcn_tran['Monto_cuota_siguiente'] = grrcn_tran['35']/100
grrcn_tran['Monto_servicio'] = grrcn_tran['39']/100
grrcn_tran['Monto_aceleracion'] = grrcn_tran['40']/100
grrcn_tran['last4_bank'] = grrcn_tran['no_tarjeta'].str[-4:].astype(str)
grrcn_tran['bin_bank'] = grrcn_tran['no_tarjeta'].str[:6].astype(str)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3239187280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['Monto_bruto_presentacion'] = grrcn_tran['17']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3239187280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['Monto_transaccion'] = grrcn_tran['18']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3239187280.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [50]:
grrcn_tran['INDEX_TOTAL'] = (grrcn_tran['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['Monto_transaccion'].astype(str).replace(' ', '',regex=True))
grrcn_tran['INDEX_PARCIAL'] = (grrcn_tran['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['codigo_aprobacion'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['Monto_transaccion'].astype(str).replace(' ', '',regex=True))
grrcn_tran['index_comisiones'] = ('OPTBLUE'+'_'+grrcn_tran['numero_cuota'].astype(str).str[:-2].replace(' ', '',regex=True))
grrcn_tran['index_comisiones_industria'] = (grrcn_tran['numero_establecimiento']+'_'+'OPTBLUE')
grrcn_tran['order_id'] = grrcn_tran['order_id'].astype(str).replace(' ', '',regex=True)
grrcn_tran['order_id'] = grrcn_tran['order_id'].str[-12:].astype(str)
grrcn_tran.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3854210186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['INDEX_TOTAL'] = (grrcn_tran['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3854210186.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_tran['INDEX_PARCIAL'] = (grrcn_tran['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\385421

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria
23347,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,0.00,0.00,0.00,0.00,5014,378750,378750_5014_JXU1OTYEVVGG_-4680.0,378750_5014_nan_-4680.0,OPTBLUE_0,8138464110_OPTBLUE
24758,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,0.00,0.00,0.00,0.00,2005,379858,379858_2005_XUVG6GPFQA10_-720.0,379858_2005_nan_-720.0,OPTBLUE_0,8138464110_OPTBLUE
18931,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,0.00,0.00,0.00,1206,371595,371595_1206_OIADJFVG5TXU_-1.0,371595_1206_nan_-1.0,OPTBLUE_0,8138464227_OPTBLUE
84796,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,0.00,0.00,0.00,1206,371595,371595_1206_OIADJFVG5TXU_-1.0,371595_1206_nan_-1.0,OPTBLUE_0,8138464227_OPTBLUE
31666,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,0.00,0.00,0.00,0.00,1009,373969,373969_1009_QTFYHXDALGVL_-1575.0,373969_1009_nan_-1575.0,OPTBLUE_0,8138464110_OPTBLUE


In [51]:
grrcn_tran.shape

(503320, 53)

In [52]:
grrcn_tran['index_comisiones'].unique()

array(['OPTBLUE_0', 'OPTBLUE_3', 'OPTBLUE_12', 'OPTBLUE_6', 'OPTBLUE_9',
       'OPTBLUE_18', 'OPTBLUE_24'], dtype=object)

### TRANSACTN CON BIN

In [53]:
grrcn_tran_bin = grrcn_tran.merge(bin_amex[['bin_bank','origen_tarjeta']],on='bin_bank',how='left')
grrcn_tran_bin['origen_tarjeta'].fillna('INTERNACIONAL',inplace=True)
grrcn_tran_bin.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria,origen_tarjeta
0,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,0.00,0.00,0.00,5014,378750,378750_5014_JXU1OTYEVVGG_-4680.0,378750_5014_nan_-4680.0,OPTBLUE_0,8138464110_OPTBLUE,nan
1,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,0.00,0.00,0.00,2005,379858,379858_2005_XUVG6GPFQA10_-720.0,379858_2005_nan_-720.0,OPTBLUE_0,8138464110_OPTBLUE,nan
2,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,0.00,0.00,1206,371595,371595_1206_OIADJFVG5TXU_-1.0,371595_1206_nan_-1.0,OPTBLUE_0,8138464227_OPTBLUE,nan
3,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,0.00,0.00,1206,371595,371595_1206_OIADJFVG5TXU_-1.0,371595_1206_nan_-1.0,OPTBLUE_0,8138464227_OPTBLUE,nan
4,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,0.00,0.00,0.00,1009,373969,373969_1009_QTFYHXDALGVL_-1575.0,373969_1009_nan_-1575.0,OPTBLUE_0,8138464110_OPTBLUE,nan


In [54]:
grrcn_tran_bin.shape

(503320, 54)

In [55]:
grrcn_tran_bin['origen_tarjeta'].unique()

array(['nan', 'NACIONAL', 'INTERNACIONAL'], dtype=object)

### TRANSACTN CON COMISIONES

In [56]:
grrcn_tran_comisiones_industria = grrcn_tran_bin.merge(comisiones_grrcn[['index_comisiones_industria','industria_fee',
                                                                        'tipo_tarjeta']]
                                                       ,on='index_comisiones_industria',how='left')
grrcn_tran_comisiones_industria.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria,origen_tarjeta,industria_fee,tipo_tarjeta
0,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,0.00,5014,378750,378750_5014_JXU1OTYEVVGG_-4680.0,378750_5014_nan_-4680.0,OPTBLUE_0,8138464110_OPTBLUE,nan,0.02,card_present
1,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,0.00,2005,379858,379858_2005_XUVG6GPFQA10_-720.0,379858_2005_nan_-720.0,OPTBLUE_0,8138464110_OPTBLUE,nan,0.02,card_present
2,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,1206,371595,371595_1206_OIADJFVG5TXU_-1.0,371595_1206_nan_-1.0,OPTBLUE_0,8138464227_OPTBLUE,nan,0.01,card_present
3,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,1206,371595,371595_1206_OIADJFVG5TXU_-1.0,371595_1206_nan_-1.0,OPTBLUE_0,8138464227_OPTBLUE,nan,0.01,card_present
4,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,0.00,1009,373969,373969_1009_QTFYHXDALGVL_-1575.0,373969_1009_nan_-1575.0,OPTBLUE_0,8138464110_OPTBLUE,nan,0.02,card_present


In [57]:
grrcn_tran_comisiones_industria.shape

(503320, 56)

In [58]:
grrcn_tran_final = grrcn_tran_comisiones_industria.merge(comisiones_grrcn[['index_comisiones','sobretasa']],
                                        on='index_comisiones',how='left')

grrcn_tran_final.loc[(grrcn_tran_final['origen_tarjeta'] == 'INTERNACIONAL'), "inbound_fee"] = 0.4/100

grrcn_tran_final.loc[(grrcn_tran_final['1'] == 'TRANSACTN'), "network_fee"] = 0.2/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 0) &
                      (grrcn_tran_final['Monto_transaccion'] < 1000)), "sobrerango_v0"] = 0

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 1000) &
                      (grrcn_tran_final['Monto_transaccion'] < 3000)), "sobrerango_v1"] = 0.4/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 3000) &
                      (grrcn_tran_final['Monto_transaccion'] < 10000)), "sobrerango_v2"] = 0.7/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 10000) &
                      (grrcn_tran_final['Monto_transaccion'] < 1000000)), "sobrerango_v3"] = 1.3/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < 0) &
                      (grrcn_tran_final['Monto_transaccion'] >= -1000)), "sobrerango_v0"] = 0

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < -1000) &
                      (grrcn_tran_final['Monto_transaccion'] >= -2999.99)), "sobrerango_v1"] = -0.4/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < -3000) &
                      (grrcn_tran_final['Monto_transaccion'] >= -9999.99)), "sobrerango_v2"] = -0.7/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < -10000) &
                      (grrcn_tran_final['Monto_transaccion'] >= -999999.99)), "sobrerango_v3"] = -1.3/100

grrcn_tran_final['sobretasa'].fillna(0,inplace=True)
grrcn_tran_final['inbound_fee'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v0'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v1'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v2'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v3'].fillna(0,inplace=True)

grrcn_tran_final['monto_sobrerango_v0'] =  grrcn_tran_final['sobrerango_v0']*100
grrcn_tran_final['iva_monto_sobrerango_v0'] =  grrcn_tran_final['monto_sobrerango_v0']*0.1600

grrcn_tran_final['monto_sobrerango_v1'] =  grrcn_tran_final['sobrerango_v1']*100
grrcn_tran_final['iva_monto_sobrerango_v1'] =  grrcn_tran_final['monto_sobrerango_v1']*0.1600

grrcn_tran_final['monto_sobrerango_v2'] =  grrcn_tran_final['sobrerango_v2']*100
grrcn_tran_final['iva_monto_sobrerango_v2'] =  grrcn_tran_final['monto_sobrerango_v2']*0.1600

grrcn_tran_final['monto_sobrerango_v3'] =  grrcn_tran_final['sobrerango_v3']*100
grrcn_tran_final['iva_monto_sobrerango_v3'] =  grrcn_tran_final['monto_sobrerango_v3']*0.1600

grrcn_tran_final['monto_comision_industria_fee'] =  grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['industria_fee'])
grrcn_tran_final['iva_monto_comision_industria_fee'] =  grrcn_tran_final['monto_comision_industria_fee']*0.1600

grrcn_tran_final['monto_comision_inbound_fee'] = grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['inbound_fee'])
grrcn_tran_final['iva_monto_comision_inbound_fee'] =  grrcn_tran_final['monto_comision_inbound_fee']*0.1600

grrcn_tran_final['monto_comision_network_fee'] = grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['network_fee'])
grrcn_tran_final['iva_monto_comision_network_fee'] =  grrcn_tran_final['monto_comision_network_fee']*0.1600

grrcn_tran_final.loc[(grrcn_tran_final['monto_comision_network_fee'] < 0), 'monto_comision_cancelacion_network_fee'] = grrcn_tran_final['monto_comision_network_fee']*-1
grrcn_tran_final['iva_monto_comision_cancelacion_network_fee'] =  grrcn_tran_final['monto_comision_cancelacion_network_fee']*0.1600

grrcn_tran_final.loc[(grrcn_tran_final['monto_comision_inbound_fee'] < 0), 'monto_comision_cancelacion_inbound_fee'] = grrcn_tran_final['monto_comision_inbound_fee']*-1
grrcn_tran_final['iva_monto_comision_cancelacion_inbound_fee'] =  grrcn_tran_final['monto_comision_cancelacion_inbound_fee']*0.1600

grrcn_tran_final['monto_comision_cancelacion_network_fee'].fillna(0,inplace=True)
grrcn_tran_final['iva_monto_comision_cancelacion_network_fee'].fillna(0,inplace=True)
grrcn_tran_final['monto_comision_cancelacion_inbound_fee'].fillna(0,inplace=True)
grrcn_tran_final['iva_monto_comision_cancelacion_inbound_fee'].fillna(0,inplace=True)

grrcn_tran_final['monto_comision_cancelacion'] = (grrcn_tran_final['monto_comision_cancelacion_network_fee'] +
                                                  grrcn_tran_final['monto_comision_cancelacion_inbound_fee'])
grrcn_tran_final['iva_monto_comision_cancelacion'] = (grrcn_tran_final['iva_monto_comision_cancelacion_network_fee'] +
                                                      grrcn_tran_final['iva_monto_comision_cancelacion_inbound_fee'])

grrcn_tran_final['monto_comision_cancelacion'].fillna(0,inplace=True)
grrcn_tran_final['iva_monto_comision_cancelacion'].fillna(0,inplace=True)

grrcn_tran_final['monto_sobretasa'] = grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['sobretasa'])
grrcn_tran_final['iva_monto_sobretasa'] =  grrcn_tran_final['monto_sobretasa']*0.1600

grrcn_tran_final['total_monto_sobrerango'] =  (grrcn_tran_final['monto_sobrerango_v0'] +
                                               grrcn_tran_final['monto_sobrerango_v1'] +
                                               grrcn_tran_final['monto_sobrerango_v2'] +
                                               grrcn_tran_final['monto_sobrerango_v3'])
grrcn_tran_final['total_iva_monto_sobrerango'] =  (grrcn_tran_final['iva_monto_sobrerango_v0'] +
                                                   grrcn_tran_final['iva_monto_sobrerango_v1'] +
                                                   grrcn_tran_final['iva_monto_sobrerango_v2'] +
                                                   grrcn_tran_final['iva_monto_sobrerango_v3'])

grrcn_tran_final['total_monto_comisiones'] =  (grrcn_tran_final['monto_comision_industria_fee'] +
                                               grrcn_tran_final['monto_comision_inbound_fee'] +
                                               grrcn_tran_final['monto_comision_network_fee'] +
                                               grrcn_tran_final['monto_comision_cancelacion'])
grrcn_tran_final['total_iva_monto_comisiones'] =  (grrcn_tran_final['iva_monto_comision_industria_fee'] +
                                                    grrcn_tran_final['iva_monto_comision_inbound_fee'] +
                                                    grrcn_tran_final['iva_monto_comision_network_fee'] +
                                                    grrcn_tran_final['iva_monto_comision_cancelacion'])

grrcn_tran_final.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,monto_comision_cancelacion_inbound_fee,iva_monto_comision_cancelacion_inbound_fee,monto_comision_cancelacion,iva_monto_comision_cancelacion,monto_sobretasa,iva_monto_sobretasa,total_monto_sobrerango,total_iva_monto_sobrerango,total_monto_comisiones,total_iva_monto_comisiones
0,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,0.00,0.00,9.36,1.50,-0.00,-0.00,-0.70,-0.11,-74.88,-11.98
1,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,0.00,0.00,1.44,0.23,-0.00,-0.00,0.00,0.00,-11.52,-1.84
2,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.02,-0.00
3,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.02,-0.00
4,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,0.00,0.00,3.15,0.50,-0.00,-0.00,-0.40,-0.06,-25.20,-4.03


In [59]:
grrcn_tran_final.shape

(503320, 89)

In [60]:
grrcn_tran_range = grrcn_tran_final[(grrcn_tran_final['fecha_pago'] >= start_date) & (grrcn_tran_final['fecha_pago'] <= final_date)]

In [61]:
grrcn_tran_range.shape

(399969, 89)

In [62]:
# grrcn_tran.to_excel('DETALLE OPTBLUE TRANSACTION.xlsx',index=False)

### DATA PROVISION

In [63]:
start_date_prov = '2024-01-20'

In [64]:
grrcn_tran_prov = grrcn_tran[(grrcn_tran['fecha_transaccion'] >= start_date_prov)]
grrcn_tran_prov.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria
82730,TRANSACTN,7464284207,002,NaN,2024-01-22,MXN,8138464136,2024-01-19,2024-01-19,813846413640120,...,0.00,0.00,0.00,0.00,3003,370786,370786_3003_BQWQYF8PFG16_217.0,370786_3003_854977_217.0,OPTBLUE_0,8138464136_OPTBLUE
255655,TRANSACTN,8138352885,002,0190096084,2024-01-22,MXN,8138464136,2024-01-19,2024-01-19,813846413640120,...,0.00,0.00,0.00,0.00,3003,370786,370786_3003_BQWQYF8PFG16_217.0,370786_3003_854977_217.0,OPTBLUE_0,8138464136_OPTBLUE
85641,TRANSACTN,7464284207,002,NaN,2024-01-22,MXN,8138464136,2024-01-19,2024-01-19,813846413640120,...,0.00,0.00,0.00,0.00,8842,379321,379321_8842_SQSEBMNEZOVY_73.0,379321_8842_822808_73.0,OPTBLUE_0,8138464136_OPTBLUE
132142,TRANSACTN,7464284207,002,NaN,2024-01-22,MXN,8138464227,2024-01-19,2024-01-19,813846422740120,...,433.34,433.33,0.00,-33.15,8675,379911,379911_8675_EQVBLECXRJBE_1300.0,379911_8675_201275_1300.0,OPTBLUE_3,8138464227_OPTBLUE
258566,TRANSACTN,8138352885,002,0190096084,2024-01-22,MXN,8138464136,2024-01-19,2024-01-19,813846413640120,...,0.00,0.00,0.00,0.00,8842,379321,379321_8842_SQSEBMNEZOVY_73.0,379321_8842_822808_73.0,OPTBLUE_0,8138464136_OPTBLUE


In [65]:
grrcn_tran_prov.shape

(293531, 53)

In [66]:
grrcn_tran_prov.to_csv('AMEX PROVISION OPTBLUE ENERO 2024.csv',index=False)

### TXNPRICING

In [67]:
grrcn_txnp['5'] = pd.to_datetime(grrcn_txnp['5'], format='%Y%m%d').dt.date
grrcn_txnp['14'] = pd.to_datetime(grrcn_txnp['14'], format='%Y%m%d').dt.date
grrcn_txnp['5'] = pd.to_datetime(grrcn_txnp['5'])
grrcn_txnp['14'] = pd.to_datetime(grrcn_txnp['14'])
grrcn_txnp.sort_values(by=['5','14'],inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1852858643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_txnp['5'] = pd.to_datetime(grrcn_txnp['5'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1852858643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_txnp['14'] = pd.to_datetime(grrcn_txnp['14'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1852858643.py:3: SettingWithCopyWarning: 
A valu

In [68]:
grrcn_txnp.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','12':'no_tarjeta','14':'fecha_transaccion'
                           ,'10':'order_id'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2912390374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_txnp.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','12':'no_tarjeta','14':'fecha_transaccion'


In [69]:
grrcn_txnp['13'] = grrcn_txnp['13'].astype(float)
grrcn_txnp['17'] = grrcn_txnp['17'].astype(float)
grrcn_txnp['18'] = grrcn_txnp['18'].astype(float)
grrcn_txnp['19'] = grrcn_txnp['19'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1531175176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_txnp['13'] = grrcn_txnp['13'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1531175176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_txnp['17'] = grrcn_txnp['17'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1531175176.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [70]:
grrcn_txnp['Monto_transaccion'] = grrcn_txnp['13']/100
grrcn_txnp['Monto_servicio'] = grrcn_txnp['17']/1000000
grrcn_txnp['Tasa_descuento'] = grrcn_txnp['18']/100000
grrcn_txnp['Monto_descuento'] = grrcn_txnp['19']/1000000
grrcn_txnp['order_id'] = grrcn_txnp['order_id'].astype(str).replace(' ', '',regex=True)
grrcn_txnp['order_id'] = grrcn_txnp['order_id'].str[-12:].astype(str)
grrcn_txnp.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1599299329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_txnp['Monto_transaccion'] = grrcn_txnp['13']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1599299329.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_txnp['Monto_servicio'] = grrcn_txnp['17']/1000000
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1599299329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,1,2,3,4,fecha_pago,6,numero_establecimiento,8,9,order_id,...,36,37,38,39,40,41,Monto_transaccion,Monto_servicio,Tasa_descuento,Monto_descuento
4955,TXNPRICING,8138352885,002,3630036365,2024-01-02,MXN,8138464219,NaN,NaN,XPQLGFSUMFG7,...,NaN,NaN,NaN,NaN,NaN,NaN,"-5,000.00",0.00,0.00,-20.00
4956,TXNPRICING,8138352885,002,3630036365,2024-01-02,MXN,8138464219,NaN,NaN,XPQLGFSUMFG7,...,NaN,NaN,NaN,NaN,NaN,NaN,"-5,000.00",0.00,-0.16,-3.20
4957,TXNPRICING,8138352885,002,3630036365,2024-01-02,MXN,8138464219,NaN,NaN,XPQLGFSUMFG7,...,NaN,NaN,NaN,NaN,NaN,NaN,"-5,000.00",0.00,0.00,-10.00
4958,TXNPRICING,8138352885,002,3630036365,2024-01-02,MXN,8138464219,NaN,NaN,XPQLGFSUMFG7,...,NaN,NaN,NaN,NaN,NaN,NaN,"-5,000.00",0.00,-0.16,-1.60
4959,TXNPRICING,8138352885,002,3630036365,2024-01-02,MXN,8138464219,NaN,NaN,XPQLGFSUMFG7,...,NaN,NaN,NaN,NaN,NaN,NaN,"-5,000.00",0.00,0.02,-80.00


In [71]:
grrcn_txnp.shape

(2634938, 45)

In [72]:
grrcn_txnp['fecha_pago'].unique()

<DatetimeArray>
['2024-01-02 00:00:00', '2024-01-03 00:00:00', '2024-01-04 00:00:00',
 '2024-01-05 00:00:00', '2024-01-08 00:00:00', '2024-01-09 00:00:00',
 '2024-01-10 00:00:00', '2024-01-11 00:00:00', '2024-01-12 00:00:00',
 '2024-01-15 00:00:00', '2024-01-16 00:00:00', '2024-01-17 00:00:00',
 '2024-01-18 00:00:00', '2024-01-19 00:00:00', '2024-01-22 00:00:00',
 '2024-01-23 00:00:00', '2024-01-24 00:00:00', '2024-01-25 00:00:00',
 '2024-01-26 00:00:00', '2024-01-29 00:00:00', '2024-01-30 00:00:00',
 '2024-01-31 00:00:00', '2024-02-01 00:00:00', '2024-02-02 00:00:00',
 '2024-02-06 00:00:00']
Length: 25, dtype: datetime64[ns]

In [73]:
grrcn_txnp_range_1 = grrcn_txnp[(grrcn_txnp['fecha_pago'] >= start_date) & (grrcn_txnp['fecha_pago'] <= final_date_1)]
grrcn_txnp_range_2 = grrcn_txnp[(grrcn_txnp['fecha_pago'] >= start_date_1) & (grrcn_txnp['fecha_pago'] <= final_date_2)]
grrcn_txnp_range_3 = grrcn_txnp[(grrcn_txnp['fecha_pago'] >= start_date_2) & (grrcn_txnp['fecha_pago'] <= final_date_3)]
grrcn_txnp_range_4 = grrcn_txnp[(grrcn_txnp['fecha_pago'] >= start_date_3) & (grrcn_txnp['fecha_pago'] <= final_date)]

In [74]:
grrcn_txnp_range_1.shape

(538502, 45)

In [75]:
grrcn_txnp_range_2.shape

(506414, 45)

In [76]:
grrcn_txnp_range_3.shape

(520692, 45)

In [77]:
grrcn_txnp_range_4.shape

(534912, 45)

### ADJUSTMENTS

In [78]:
grrcn_adju['5'] = pd.to_datetime(grrcn_adju['5'], format='%Y%m%d').dt.date
grrcn_adju['8'] = pd.to_datetime(grrcn_adju['8'], format='%Y%m%d').dt.date
grrcn_adju['14'] = pd.to_datetime(grrcn_adju['14'], format='%Y%m%d').dt.date
grrcn_adju['5'] = pd.to_datetime(grrcn_adju['5'])
grrcn_adju['8'] = pd.to_datetime(grrcn_adju['8'])
grrcn_adju['14'] = pd.to_datetime(grrcn_adju['14'])
grrcn_adju.sort_values(by=['5'],inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1514995302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_adju['5'] = pd.to_datetime(grrcn_adju['5'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1514995302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_adju['8'] = pd.to_datetime(grrcn_adju['8'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1514995302.py:3: SettingWithCopyWarning: 
A value 

In [79]:
grrcn_adju.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_envio','12':'no_tarjeta'
                           ,'14':'fecha_procesamiento','17':'numero_ajuste','18':'codigo_ajuste'
                           ,'19':'motivo_ajuste'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2325207570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_adju.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_envio','12':'no_tarjeta'


In [80]:
grrcn_adju['20'] = grrcn_adju['20'].astype(float)
grrcn_adju['21'] = grrcn_adju['21'].astype(float)
grrcn_adju['22'] = grrcn_adju['22'].astype(float)
grrcn_adju['23'] = grrcn_adju['23'].astype(float)
grrcn_adju['24'] = grrcn_adju['24'].astype(float)
grrcn_adju['25'] = grrcn_adju['25'].astype(float)
grrcn_adju['26'] = grrcn_adju['26'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2929771404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_adju['20'] = grrcn_adju['20'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2929771404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_adju['21'] = grrcn_adju['21'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2929771404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [81]:
grrcn_adju['Monto_bruto_adjustment'] = grrcn_adju['20']/100
grrcn_adju['Monto_descuento_adjustment'] = grrcn_adju['21']/100
grrcn_adju['Monto_servicio_adjustment'] = grrcn_adju['22']/100
grrcn_adju['Monto_impuesto_adjustment'] = grrcn_adju['23']/100
grrcn_adju['Monto_neto_adjustment'] = grrcn_adju['24']/100
grrcn_adju['Tasa_descuento_adjustment'] = grrcn_adju['25']/100
grrcn_adju['Tasa_servicio_adjustment'] = grrcn_adju['26']/100
grrcn_adju.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\23421778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_adju['Monto_bruto_adjustment'] = grrcn_adju['20']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\23421778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_adju['Monto_descuento_adjustment'] = grrcn_adju['21']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\23421778.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_envio,9,10,...,39,40,41,Monto_bruto_adjustment,Monto_descuento_adjustment,Monto_servicio_adjustment,Monto_impuesto_adjustment,Monto_neto_adjustment,Tasa_descuento_adjustment,Tasa_servicio_adjustment
229710,ADJUSTMENT,8138352885,002,3630036365,2024-01-02,MXN,8138464193,2023-12-29,NaN,NaN,...,NaN,NaN,NaN,0.00,0.00,"15,697.14","2,511.54","-18,208.68",NaN,0.00
229704,ADJUSTMENT,8138352885,002,3630036365,2024-01-02,MXN,8138464177,2023-12-29,NaN,NaN,...,NaN,NaN,NaN,0.00,0.00,"3,691.27",590.60,"-4,281.87",NaN,0.00
229703,ADJUSTMENT,8138352885,002,3630036365,2024-01-02,MXN,8138464144,2023-12-29,NaN,NaN,...,NaN,NaN,NaN,0.00,0.00,"8,958.40","1,433.34","-10,391.74",NaN,0.00
229702,ADJUSTMENT,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,NaN,NaN,...,NaN,NaN,NaN,0.00,0.00,"3,685.20",589.63,"-4,274.83",NaN,0.00
229701,ADJUSTMENT,8138352885,002,3630036365,2024-01-02,MXN,8138464110,2023-12-29,NaN,NaN,...,NaN,NaN,NaN,0.00,0.00,824.75,131.96,-956.71,NaN,0.00


In [82]:
grrcn_adju.shape

(326, 48)

In [83]:
grrcn_adju_range = grrcn_adju[(grrcn_adju['fecha_pago'] >= start_date) & (grrcn_adju['fecha_pago'] <= final_date)]

### CHARGEBACKS

In [84]:
grrcn_char['5'] = pd.to_datetime(grrcn_char['5'], format='%Y%m%d').dt.date
grrcn_char['8'] = pd.to_datetime(grrcn_char['8'], format='%Y%m%d').dt.date
grrcn_char['14'] = pd.to_datetime(grrcn_char['14'], format='%Y%m%d').dt.date
grrcn_char['5'] = pd.to_datetime(grrcn_char['5'])
grrcn_char['8'] = pd.to_datetime(grrcn_char['8'])
grrcn_char['14'] = pd.to_datetime(grrcn_char['14'])
grrcn_char.sort_values(by=['5'],inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1249772799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_char['5'] = pd.to_datetime(grrcn_char['5'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1249772799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_char['8'] = pd.to_datetime(grrcn_char['8'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1249772799.py:3: SettingWithCopyWarning: 
A value 

In [85]:
grrcn_char.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_presentacion','10':'order_id'
                           ,'11':'identificador_vendedor','12':'numero_tarjeta','14':'fecha_procesamiento'
                           ,'17':'numero_contracargo','18':'codigo_contracargo','19':'descripcion_contracargo'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\4147288204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_char.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_presentacion','10':'order_id'


In [86]:
grrcn_char['20'] = grrcn_char['20'].astype(float)
grrcn_char['21'] = grrcn_char['21'].astype(float)
grrcn_char['22'] = grrcn_char['22'].astype(float)
grrcn_char['23'] = grrcn_char['23'].astype(float)
grrcn_char['24'] = grrcn_char['24'].astype(float)
grrcn_char['25'] = grrcn_char['25'].astype(float)
grrcn_char['26'] = grrcn_char['26'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3040258524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_char['20'] = grrcn_char['20'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3040258524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_char['21'] = grrcn_char['21'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3040258524.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [87]:
grrcn_char['Monto_bruto_contracargo'] = grrcn_char['20']/100
grrcn_char['Monto_descuento_contracargo'] = grrcn_char['21']/100
grrcn_char['Monto_servicio_contracargo'] = grrcn_char['22']/100
grrcn_char['Monto_impuesto_contracargo'] = grrcn_char['23']/100
grrcn_char['Monto_neto_contracargo'] = grrcn_char['24']/100
grrcn_char['Tasa_descuento_contracargo'] = grrcn_char['25']/100
grrcn_char['Tasa_servicio_contracargo'] = grrcn_char['26']/100
grrcn_char['order_id'] = grrcn_char['order_id'].astype(str).replace(' ', '',regex=True)
grrcn_char['order_id'] = grrcn_char['order_id'].str[-12:].astype(str)
grrcn_char.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3950269377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_char['Monto_bruto_contracargo'] = grrcn_char['20']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3950269377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grrcn_char['Monto_descuento_contracargo'] = grrcn_char['21']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3950269377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,9,order_id,...,39,40,41,Monto_bruto_contracargo,Monto_descuento_contracargo,Monto_servicio_contracargo,Monto_impuesto_contracargo,Monto_neto_contracargo,Tasa_descuento_contracargo,Tasa_servicio_contracargo
229670,CHARGEBACK,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,NaN,ZB2PYA0CZNAP,...,NaN,NaN,NaN,"-6,417.00",0.00,0.00,0.00,"-6,417.00",NaN,0.00
229667,CHARGEBACK,8138352885,002,3630036365,2024-01-02,MXN,8138464128,2023-12-29,NaN,XJLGD7HQ7HP9,...,NaN,NaN,NaN,880.00,0.00,0.00,0.00,880.00,NaN,0.00
229666,CHARGEBACK,8138352885,002,3630036365,2024-01-02,MXN,8138464110,2023-12-29,NaN,773PIBU6BK7J,...,NaN,NaN,NaN,"-1,021.44",0.00,0.00,0.00,"-1,021.44",NaN,0.00
229665,CHARGEBACK,8138352885,002,3630036365,2024-01-02,MXN,8138464227,2023-12-29,NaN,UOK7KBGJC1QH,...,NaN,NaN,NaN,"-1,573.39",0.00,0.00,0.00,"-1,573.39",NaN,0.00
229664,CHARGEBACK,8138352885,002,3630036365,2024-01-02,MXN,8138464227,2023-12-29,NaN,7ZZGLTGZ7YMU,...,NaN,NaN,NaN,"-4,952.80",0.00,0.00,0.00,"-4,952.80",NaN,0.00


In [88]:
grrcn_char.shape

(409, 48)

In [89]:
grrcn_char_range = grrcn_char[(grrcn_char['fecha_pago'] >= start_date) & (grrcn_char['fecha_pago'] <= final_date)]

# CONCILIACION ALL

In [90]:
cruce_1_previo = grrcn_tran_final.merge(db[['order_id','transaction_id','clave_autorizacion','date_transaction','amount',
                                            'status_code','msi','bin','last4','affiliation','mcc','affiliation_type']],
                                        on='order_id',how='left')
cruce_1_previo[['transaction_id','status_code','msi']].fillna('NO ENCONTRADO',inplace=True)
cruce_1_previo['amount'].fillna(0,inplace=True)
cruce_1_previo['status_code'].fillna('NO ENCONTRADO',inplace=True)
cruce_1_previo.loc[~cruce_1_previo['status_code'].str.contains('3'),'amount_new'] = cruce_1_previo['amount']
cruce_1_previo.loc[cruce_1_previo['status_code'].str.contains('3'),'amount_new'] = cruce_1_previo['amount']*-1
cruce_1_previo['comparison'] = cruce_1_previo['Monto_transaccion'] == cruce_1_previo['amount_new']
cruce_1_previo.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3755919643.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_previo[['transaction_id','status_code','msi']].fillna('NO ENCONTRADO',inplace=True)


,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
0,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,"4,680.00",2,0.00000,378750,5014,8138464110,4722,OPTBLUE,"4,680.00",False
1,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,720.00,2,0.00000,379858,2005,8138464110,4722,OPTBLUE,720.00,False
2,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
3,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
4,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,"1,575.00",2,0.00000,373969,1009,8138464110,4722,OPTBLUE,"1,575.00",False


In [91]:
cruce_1_previo.shape

(503474, 102)

In [92]:
cruce_1_previo['status_code'].unique()

array(['2', '-6', '1', '3', '8', 'NO ENCONTRADO'], dtype=object)

### FILTRAR LOS RECHAZADOS AMEX

In [93]:
cruce_1 = cruce_1_previo[(cruce_1_previo['rechazo_amex'] != 'REJ')]
cruce_1.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
0,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,"4,680.00",2,0.00000,378750,5014,8138464110,4722,OPTBLUE,"4,680.00",False
1,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,720.00,2,0.00000,379858,2005,8138464110,4722,OPTBLUE,720.00,False
2,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
3,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
4,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,"1,575.00",2,0.00000,373969,1009,8138464110,4722,OPTBLUE,"1,575.00",False


In [94]:
cruce_1.shape

(503474, 102)

### RECHAZADOS AMEX

In [95]:
cruce_1_rej = cruce_1_previo[(cruce_1_previo['rechazo_amex'] == 'REJ')]
cruce_1_rej.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison


In [96]:
cruce_1_rej.shape

(0, 102)

# PAYMENTS CRUCE GRRCN VS CLIP

In [97]:
txn_grrcn_deposit = cruce_1[(cruce_1['fecha_pago'] >= start_date) & (cruce_1['fecha_pago'] <= final_date)]
txn_grrcn_deposit.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
0,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,"4,680.00",2,0.00000,378750,5014,8138464110,4722,OPTBLUE,"4,680.00",False
1,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,720.00,2,0.00000,379858,2005,8138464110,4722,OPTBLUE,720.00,False
2,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
3,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
4,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,"1,575.00",2,0.00000,373969,1009,8138464110,4722,OPTBLUE,"1,575.00",False


In [98]:
txn_grrcn_deposit.shape

(400111, 102)

In [99]:
txn_grrcn_deposit['fecha_pago'].unique()

<DatetimeArray>
['2024-01-03 00:00:00', '2024-01-04 00:00:00', '2024-01-18 00:00:00',
 '2024-01-11 00:00:00', '2024-01-22 00:00:00', '2024-01-02 00:00:00',
 '2024-01-12 00:00:00', '2024-01-09 00:00:00', '2024-01-08 00:00:00',
 '2024-01-05 00:00:00', '2024-01-15 00:00:00', '2024-01-19 00:00:00',
 '2024-01-10 00:00:00', '2024-01-16 00:00:00', '2024-01-17 00:00:00',
 '2024-01-26 00:00:00', '2024-01-23 00:00:00', '2024-01-24 00:00:00',
 '2024-01-31 00:00:00', '2024-01-29 00:00:00', '2024-01-25 00:00:00',
 '2024-01-30 00:00:00']
Length: 22, dtype: datetime64[ns]

In [100]:
txn_grrcn_deposit.columns

Index(['1', '2', '3', '4', 'fecha_pago', '6', 'numero_establecimiento',
       'fecha_presentacion', 'fecha_procesamiento', '10',
       ...
       'amount', 'status_code', 'msi', 'bin', 'last4', 'affiliation', 'mcc',
       'affiliation_type', 'amount_new', 'comparison'],
      dtype='object', length=102)

In [101]:
duplicates_1 = txn_grrcn_deposit[txn_grrcn_deposit.duplicated(['order_id'] , keep=False)]
duplicates_1.sort_values(by=['order_id','fecha_transaccion'],inplace=True)
duplicates_1.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1230631047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates_1.sort_values(by=['order_id','fecha_transaccion'],inplace=True)


,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
295211,TRANSACTN,7464284207,002,NaN,2024-01-26,MXN,8138464136,2024-01-25,2024-01-25,813846413640125,...,418.00,1,0.00000,341161,1017,8138464136,5814,OPTBLUE,418.00,True
295212,TRANSACTN,8138352885,002,0250054405,2024-01-26,MXN,8138464136,2024-01-25,2024-01-25,813846413640125,...,418.00,1,0.00000,341161,1017,8138464136,5814,OPTBLUE,418.00,True
162922,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464136,2024-01-17,2024-01-17,813846413640117,...,40.00,1,0.00000,371777,2002,8138464136,5812,OPTBLUE,40.00,True
162923,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464136,2024-01-17,2024-01-17,813846413640117,...,40.00,1,0.00000,371777,2002,8138464136,5812,OPTBLUE,40.00,True
316991,TRANSACTN,7464284207,002,NaN,2024-01-29,MXN,8138464136,2024-01-26,2024-01-26,813846413640126,...,28.00,1,0.00000,371775,1020,8138464136,5812,OPTBLUE,28.00,True


In [102]:
duplicates_1.shape

(239376, 102)

### COMPARISON

In [103]:
comparison_1 = txn_grrcn_deposit[(txn_grrcn_deposit['comparison'] == False)]
comparison_1.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
0,TRANSACTN,8138352885,002,0020024044,2024-01-03,MXN,8138464110,2024-01-02,2024-01-02,813846411040102,...,"4,680.00",2,0.00000,378750,5014,8138464110,4722,OPTBLUE,"4,680.00",False
1,TRANSACTN,8138352885,002,0030034750,2024-01-04,MXN,8138464110,2024-01-03,2024-01-03,813846411040103,...,720.00,2,0.00000,379858,2005,8138464110,4722,OPTBLUE,720.00,False
2,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
3,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464227,2024-01-17,2024-01-17,813846422740117,...,1.00,2,0.00000,371595,1206,8138464227,5971,OPTBLUE,1.00,False
4,TRANSACTN,8138352885,002,0100004965,2024-01-11,MXN,8138464110,2024-01-10,2024-01-10,813846411040110,...,"1,575.00",2,0.00000,373969,1009,8138464110,4722,OPTBLUE,"1,575.00",False


In [104]:
comparison_1.shape

(2021, 102)

### NO ENCONTRADOS

In [105]:
not_found_txn_epa_deposit = txn_grrcn_deposit[(txn_grrcn_deposit['status_code'] == 'NO ENCONTRADO')]
not_found_txn_epa_deposit.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
125791,TRANSACTN,8138352885,002,0120026148,2024-01-15,MXN,8138464136,2024-01-12,2024-01-12,813846413640113,...,0.00,NO ENCONTRADO,NaN,NaN,NaN,NaN,NaN,NaN,0.00,False


In [106]:
not_found_txn_epa_deposit.shape

(1, 102)

### ADJUSTMENTS CONCILIACION

In [107]:
adj_montos = grrcn_adju[(grrcn_adju['fecha_pago'] >= start_date) & (grrcn_adju['fecha_pago'] <= final_date)]

In [108]:
deposit_date_adj = adj_montos.groupby(['fecha_pago']).agg({'Monto_bruto_adjustment':sum,'Monto_descuento_adjustment':sum
                                                                 ,'Monto_servicio_adjustment':sum,'Monto_impuesto_adjustment':sum
                                                                 ,'Monto_neto_adjustment':sum}).reset_index()
deposit_date_adj.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2290818785.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj = adj_montos.groupby(['fecha_pago']).agg({'Monto_bruto_adjustment':sum,'Monto_descuento_adjustment':sum


,fecha_pago,Monto_bruto_adjustment,Monto_descuento_adjustment,Monto_servicio_adjustment,Monto_impuesto_adjustment,Monto_neto_adjustment
0,2024-01-02,0.00,0.00,"245,859.55","39,337.51","-285,197.06"
1,2024-01-03,14.33,0.00,"84,205.12","13,472.83","-97,663.62"
2,2024-01-04,0.00,0.00,"122,143.88","19,543.02","-141,686.90"
3,2024-01-05,0.00,0.00,"111,358.17","17,817.30","-129,175.47"
4,2024-01-08,0.00,0.00,"288,787.71","46,206.04","-334,993.75"


In [109]:
deposit_date_adj.shape

(22, 6)

### ADJUSTMENTS POSITIVOS

In [110]:
adj_montos_positivos = adj_montos[(adj_montos['Monto_bruto_adjustment'] >= 0)]

In [111]:
deposit_date_adj_positivos = adj_montos_positivos.groupby(['fecha_pago']).agg({'Monto_bruto_adjustment':sum,'Monto_descuento_adjustment':sum
                                                                 ,'Monto_servicio_adjustment':sum,'Monto_impuesto_adjustment':sum
                                                                 ,'Monto_neto_adjustment':sum,}).reset_index()
deposit_date_adj_positivos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3447209873.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj_positivos = adj_montos_positivos.groupby(['fecha_pago']).agg({'Monto_bruto_adjustment':sum,'Monto_descuento_adjustment':sum


,fecha_pago,Monto_bruto_adjustment,Monto_descuento_adjustment,Monto_servicio_adjustment,Monto_impuesto_adjustment,Monto_neto_adjustment
0,2024-01-02,0.00,0.00,"245,859.55","39,337.51","-285,197.06"
1,2024-01-03,14.33,0.00,"84,205.12","13,472.83","-97,663.62"
2,2024-01-04,0.00,0.00,"122,143.88","19,543.02","-141,686.90"
3,2024-01-05,0.00,0.00,"111,358.17","17,817.30","-129,175.47"
4,2024-01-08,0.00,0.00,"288,787.71","46,206.04","-334,993.75"


In [112]:
deposit_date_adj_positivos.shape

(22, 6)

### ADJUSTMENTS NEGATIVOS

In [113]:
adj_montos_negativos = adj_montos[(adj_montos['Monto_bruto_adjustment'] < 0)]

In [114]:
deposit_date_adj_negativos = adj_montos_negativos.groupby(['fecha_pago']).agg({'Monto_bruto_adjustment':sum,'Monto_descuento_adjustment':sum
                                                                 ,'Monto_servicio_adjustment':sum,'Monto_impuesto_adjustment':sum
                                                                 ,'Monto_neto_adjustment':sum,}).reset_index()
deposit_date_adj_negativos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1749769636.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj_negativos = adj_montos_negativos.groupby(['fecha_pago']).agg({'Monto_bruto_adjustment':sum,'Monto_descuento_adjustment':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1749769636.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj_negativos = adj_montos_negativos.groupby(['fecha_pago']).agg({'Monto_bruto_adjustment':sum,'Monto_descuento_adjustment':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1749769636.py:1: FutureWarning: 

,fecha_pago,Monto_bruto_adjustment,Monto_descuento_adjustment,Monto_servicio_adjustment,Monto_impuesto_adjustment,Monto_neto_adjustment


In [115]:
deposit_date_adj_negativos.shape

(0, 6)

### CHARGEBACKS CONCILIACION

In [116]:
chbk_montos = grrcn_char[(grrcn_char['fecha_pago'] >= start_date) & (grrcn_char['fecha_pago'] <= final_date)]

In [117]:
deposit_date_chbk = chbk_montos.groupby(['fecha_pago']).agg({'Monto_bruto_contracargo':sum,'Monto_descuento_contracargo':sum
                                                                 ,'Monto_servicio_contracargo':sum,'Monto_impuesto_contracargo':sum
                                                                 ,'Monto_neto_contracargo':sum}).reset_index()
deposit_date_chbk.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\864583161.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_chbk = chbk_montos.groupby(['fecha_pago']).agg({'Monto_bruto_contracargo':sum,'Monto_descuento_contracargo':sum


,fecha_pago,Monto_bruto_contracargo,Monto_descuento_contracargo,Monto_servicio_contracargo,Monto_impuesto_contracargo,Monto_neto_contracargo
0,2024-01-02,"-368,623.64",0.00,0.00,0.00,"-368,623.64"
1,2024-01-03,"-144,357.74",0.00,0.00,0.00,"-144,357.74"
2,2024-01-04,"-432,603.62",0.00,0.00,0.00,"-432,603.62"
3,2024-01-05,"-77,597.03",0.00,0.00,0.00,"-77,597.03"
4,2024-01-08,"-83,767.93",0.00,0.00,0.00,"-83,767.93"


In [118]:
deposit_date_chbk.shape

(21, 6)

### CHARGEBACKS POSITIVOS

In [119]:
chbk_montos_positivos = chbk_montos[(chbk_montos['Monto_bruto_contracargo'] >= 0)]

In [120]:
deposit_date_chnk_positivos = chbk_montos_positivos.groupby(['fecha_pago']).agg({'Monto_bruto_contracargo':sum
                                                                                 ,'Monto_descuento_contracargo':sum
                                                                                 ,'Monto_servicio_contracargo':sum
                                                                                 ,'Monto_impuesto_contracargo':sum
                                                                                 ,'Monto_neto_contracargo':sum}).reset_index()
deposit_date_chnk_positivos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2434575695.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_chnk_positivos = chbk_montos_positivos.groupby(['fecha_pago']).agg({'Monto_bruto_contracargo':sum


,fecha_pago,Monto_bruto_contracargo,Monto_descuento_contracargo,Monto_servicio_contracargo,Monto_impuesto_contracargo,Monto_neto_contracargo
0,2024-01-02,"1,530.00",0.00,0.00,0.00,"1,530.00"
1,2024-01-03,"58,018.21",0.00,0.00,0.00,"58,018.21"
2,2024-01-04,"7,032.66",0.00,0.00,0.00,"7,032.66"
3,2024-01-05,"1,572.00",0.00,0.00,0.00,"1,572.00"
4,2024-01-09,"2,394.17",0.00,0.00,0.00,"2,394.17"


In [121]:
deposit_date_chnk_positivos.shape

(17, 6)

### CHARGEBACKS NEGATIVOS

In [122]:
chbk_montos_negativos = chbk_montos[(chbk_montos['Monto_bruto_contracargo'] < 0)]

In [123]:
deposit_date_chnk_negativos = chbk_montos_negativos.groupby(['fecha_pago']).agg({'Monto_bruto_contracargo':sum
                                                                                 ,'Monto_descuento_contracargo':sum
                                                                                 ,'Monto_servicio_contracargo':sum
                                                                                 ,'Monto_impuesto_contracargo':sum
                                                                                 ,'Monto_neto_contracargo':sum}).reset_index()
deposit_date_chnk_negativos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2035809174.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_chnk_negativos = chbk_montos_negativos.groupby(['fecha_pago']).agg({'Monto_bruto_contracargo':sum


,fecha_pago,Monto_bruto_contracargo,Monto_descuento_contracargo,Monto_servicio_contracargo,Monto_impuesto_contracargo,Monto_neto_contracargo
0,2024-01-02,"-370,153.64",0.00,0.00,0.00,"-370,153.64"
1,2024-01-03,"-202,375.95",0.00,0.00,0.00,"-202,375.95"
2,2024-01-04,"-439,636.28",0.00,0.00,0.00,"-439,636.28"
3,2024-01-05,"-79,169.03",0.00,0.00,0.00,"-79,169.03"
4,2024-01-08,"-83,767.93",0.00,0.00,0.00,"-83,767.93"


In [124]:
deposit_date_chnk_negativos.shape

(21, 6)

# GROUP BY DEPOSIT DATE

### POR FECHA DE DEPOSITO

In [125]:
deposit_date_cruce_1 = txn_grrcn_deposit.groupby(['fecha_pago']).agg({'transaction_id':'count',
                                                                      'Monto_transaccion':sum,
                                                                      'Monto_primera_cuota':sum,
                                                                      'Monto_cuota_siguiente':sum,
                                                                      'Monto_servicio':sum,
                                                                      'Monto_aceleracion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_date_cruce_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\661123068.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_cruce_1 = txn_grrcn_deposit.groupby(['fecha_pago']).agg({'transaction_id':'count',


,fecha_pago,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,...,iva_monto_comision_network_fee,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa
0,2024-01-02,34464,"76,331,831.87","840,300.77","840,290.31",0.00,"-245,859.55",0.00,"3,533.60","2,785.30",...,"24,426.19",468.55,"204,380.34",16.94,"8,031.00","1,433,075.10","236,687.04","1,284.96","229,292.02","37,869.93"
1,2024-01-03,7217,"17,124,058.15","276,109.11","276,106.72",0.00,"-84,205.12",0.00,702.00,547.40,...,"5,479.70",62.32,"44,120.97",11.16,"1,657.60","310,463.39","83,492.35",265.22,"49,674.14","13,358.78"
2,2024-01-04,8075,"19,897,396.67","278,762.83","278,760.04",0.00,"-122,143.88",0.00,784.80,622.30,...,"6,367.17",89.07,"49,516.40",0.69,"1,869.90","349,833.29","93,390.95",299.18,"55,973.33","14,942.55"
3,2024-01-05,8201,"20,169,052.44","292,423.89","292,420.98",0.00,"-111,358.17",0.00,773.20,639.80,...,"6,454.10",119.34,"50,177.60",11.52,"1,884.90","354,765.93","103,196.08",301.58,"56,762.55","16,511.37"
4,2024-01-08,24735,"55,993,206.29","683,493.12","683,484.02",0.00,"-288,787.71",0.00,"2,260.00","1,653.40",...,"17,917.83",354.82,"149,335.47",22.02,"5,065.20","1,047,688.32","267,498.52",810.43,"167,630.13","42,799.76"


In [126]:
#industria_fee	network_fee	sobretasa	inbound_fee	sobrerango_v2	sobrerango_v3	sobrerango_v1	sobrerango_v0

In [127]:
deposit_date_cruce_1.shape

(22, 29)

### CRUCE POR MESES SIN INTERESES

In [128]:
deposit_date_msi = txn_grrcn_deposit.groupby(['numero_cuota','fecha_pago']).agg({'transaction_id':'count',
                                                                                 'Monto_transaccion':sum,
                                                                                 'amount':sum,
                                                                                 'monto_sobretasa':sum,
                                                                                 'iva_monto_sobretasa':sum}).reset_index()
deposit_date_msi.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2653738255.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_msi = txn_grrcn_deposit.groupby(['numero_cuota','fecha_pago']).agg({'transaction_id':'count',


,numero_cuota,fecha_pago,transaction_id,Monto_transaccion,amount,monto_sobretasa,iva_monto_sobretasa
0,0.00,2024-01-02,33936,"72,021,292.65","72,125,853.36",0.00,0.00
1,0.00,2024-01-03,7087,"15,698,626.16","15,814,241.58",0.00,0.00
2,0.00,2024-01-04,7917,"18,194,930.92","18,213,500.71",0.00,0.00
3,0.00,2024-01-05,8035,"18,504,232.29","18,615,088.41",0.00,0.00
4,0.00,2024-01-08,24281,"51,879,794.93","52,114,418.59",0.00,0.00


In [129]:
deposit_date_msi.shape

(150, 7)

### CRUCE CON CONTRACARGOS

In [130]:
deposit_date_cruce_2 = deposit_date_cruce_1.merge(deposit_date_chbk[['fecha_pago','Monto_bruto_contracargo'
                                                                     ,'Monto_descuento_contracargo','Monto_servicio_contracargo'
                                                                     ,'Monto_impuesto_contracargo','Monto_neto_contracargo']]
                                                  ,on='fecha_pago',how='left')
deposit_date_cruce_2['Monto_bruto_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_descuento_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_servicio_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_impuesto_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_neto_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2.head()

,fecha_pago,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,...,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,Monto_bruto_contracargo,Monto_descuento_contracargo,Monto_servicio_contracargo,Monto_impuesto_contracargo,Monto_neto_contracargo
0,2024-01-02,34464,"76,331,831.87","840,300.77","840,290.31",0.00,"-245,859.55",0.00,"3,533.60","2,785.30",...,"1,433,075.10","236,687.04","1,284.96","229,292.02","37,869.93","-368,623.64",0.00,0.00,0.00,"-368,623.64"
1,2024-01-03,7217,"17,124,058.15","276,109.11","276,106.72",0.00,"-84,205.12",0.00,702.00,547.40,...,"310,463.39","83,492.35",265.22,"49,674.14","13,358.78","-144,357.74",0.00,0.00,0.00,"-144,357.74"
2,2024-01-04,8075,"19,897,396.67","278,762.83","278,760.04",0.00,"-122,143.88",0.00,784.80,622.30,...,"349,833.29","93,390.95",299.18,"55,973.33","14,942.55","-432,603.62",0.00,0.00,0.00,"-432,603.62"
3,2024-01-05,8201,"20,169,052.44","292,423.89","292,420.98",0.00,"-111,358.17",0.00,773.20,639.80,...,"354,765.93","103,196.08",301.58,"56,762.55","16,511.37","-77,597.03",0.00,0.00,0.00,"-77,597.03"
4,2024-01-08,24735,"55,993,206.29","683,493.12","683,484.02",0.00,"-288,787.71",0.00,"2,260.00","1,653.40",...,"1,047,688.32","267,498.52",810.43,"167,630.13","42,799.76","-83,767.93",0.00,0.00,0.00,"-83,767.93"


In [131]:
deposit_date_cruce_2.shape

(22, 34)

### CRUCE CON AJUSTES

In [132]:
deposit_date_cruce_3 = deposit_date_cruce_2.merge(deposit_date_adj[['fecha_pago',
                                                                     'Monto_bruto_adjustment',
                                                                     'Monto_descuento_adjustment',
                                                                    'Monto_servicio_adjustment',
                                                                    'Monto_impuesto_adjustment',
                                                                    'Monto_neto_adjustment']],
                                                  on='fecha_pago',how='left')
deposit_date_cruce_3['Monto_bruto_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_descuento_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_servicio_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_impuesto_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_neto_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_ajustes'] = (deposit_date_cruce_3['Monto_neto_adjustment'] +
                                             deposit_date_cruce_3['Monto_neto_contracargo'])
deposit_date_cruce_3['Monto_ajustes_calc'] = (-deposit_date_cruce_3['monto_sobretasa'] +
                                              -deposit_date_cruce_3['iva_monto_sobretasa'] +
                                             deposit_date_cruce_3['Monto_neto_contracargo'])
deposit_date_cruce_3['Monto_impuesto_calc'] = (deposit_date_cruce_3['total_iva_monto_sobrerango'] +
                                              deposit_date_cruce_3['total_iva_monto_comisiones'])
deposit_date_cruce_3['Monto_neto_calc'] = (deposit_date_cruce_3['Monto_transaccion'] -
                                          deposit_date_cruce_3['total_monto_sobrerango'] -
                                          deposit_date_cruce_3['total_monto_comisiones'] -
                                          deposit_date_cruce_3['Monto_impuesto_calc'] +
                                          deposit_date_cruce_3['Monto_ajustes_calc'])
deposit_date_cruce_3.head()

,fecha_pago,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,...,Monto_neto_contracargo,Monto_bruto_adjustment,Monto_descuento_adjustment,Monto_servicio_adjustment,Monto_impuesto_adjustment,Monto_neto_adjustment,Monto_ajustes,Monto_ajustes_calc,Monto_impuesto_calc,Monto_neto_calc
0,2024-01-02,34464,"76,331,831.87","840,300.77","840,290.31",0.00,"-245,859.55",0.00,"3,533.60","2,785.30",...,"-368,623.64",0.00,0.00,"245,859.55","39,337.51","-285,197.06","-653,820.70","-643,180.61","230,576.98","74,016,968.18"
1,2024-01-03,7217,"17,124,058.15","276,109.11","276,106.72",0.00,"-84,205.12",0.00,702.00,547.40,...,"-144,357.74",14.33,0.00,"84,205.12","13,472.83","-97,663.62","-242,021.36","-241,208.87","49,939.36","16,520,788.93"
2,2024-01-04,8075,"19,897,396.67","278,762.83","278,760.04",0.00,"-122,143.88",0.00,784.80,622.30,...,"-432,603.62",0.00,0.00,"122,143.88","19,543.02","-141,686.90","-574,290.52","-540,937.12","56,272.51","18,948,483.86"
3,2024-01-05,8201,"20,169,052.44","292,423.89","292,420.98",0.00,"-111,358.17",0.00,773.20,639.80,...,"-77,597.03",0.00,0.00,"111,358.17","17,817.30","-129,175.47","-206,772.50","-197,304.48","57,064.13","19,558,033.00"
4,2024-01-08,24735,"55,993,206.29","683,493.12","683,484.02",0.00,"-288,787.71",0.00,"2,260.00","1,653.40",...,"-83,767.93",0.00,0.00,"288,787.71","46,206.04","-334,993.75","-418,761.68","-394,066.21","168,440.56","54,377,946.00"


In [133]:
deposit_date_cruce_3.shape

(22, 43)

### CRUCE FINAL DESGLOSADO

In [134]:
deposit_date_cruce_final_desg = deposit_date_cruce_3[['fecha_pago','transaction_id','Monto_transaccion','monto_sobrerango_v0','monto_sobrerango_v1',
                                                 'monto_sobrerango_v2','monto_sobrerango_v3','monto_comision_network_fee',
                                                 'monto_comision_inbound_fee','monto_comision_industria_fee',
                                                 'monto_comision_cancelacion','iva_monto_sobrerango_v0',
                                                 'iva_monto_sobrerango_v1','iva_monto_sobrerango_v2','iva_monto_sobrerango_v3',
                                                 'iva_monto_comision_network_fee','iva_monto_comision_inbound_fee',
                                                 'iva_monto_comision_industria_fee','iva_monto_comision_cancelacion',
                                                 'total_monto_sobrerango','total_monto_comisiones','monto_sobretasa',
                                                 'total_iva_monto_sobrerango','total_iva_monto_comisiones','iva_monto_sobretasa',
                                                 'Monto_bruto_contracargo','Monto_descuento_contracargo',
                                                 'Monto_servicio_contracargo','Monto_impuesto_contracargo',
                                                 'Monto_neto_contracargo','Monto_bruto_adjustment','Monto_descuento_adjustment',
                                                 'Monto_servicio_adjustment','Monto_impuesto_adjustment','Monto_neto_adjustment',
                                                 'Monto_ajustes','Monto_ajustes_calc','Monto_impuesto_calc','Monto_neto_calc']]
deposit_date_cruce_final_desg.head()

,fecha_pago,transaction_id,Monto_transaccion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,monto_sobrerango_v3,monto_comision_network_fee,monto_comision_inbound_fee,monto_comision_industria_fee,...,Monto_neto_contracargo,Monto_bruto_adjustment,Monto_descuento_adjustment,Monto_servicio_adjustment,Monto_impuesto_adjustment,Monto_neto_adjustment,Monto_ajustes,Monto_ajustes_calc,Monto_impuesto_calc,Monto_neto_calc
0,2024-01-02,34464,"76,331,831.87",0.00,"3,533.60","2,785.30","1,712.10","152,663.66","2,928.42","1,277,377.14",...,"-368,623.64",0.00,0.00,"245,859.55","39,337.51","-285,197.06","-653,820.70","-643,180.61","230,576.98","74,016,968.18"
1,2024-01-03,7217,"17,124,058.15",0.00,702.00,547.40,408.20,"34,248.12",389.47,"275,756.05",...,"-144,357.74",14.33,0.00,"84,205.12","13,472.83","-97,663.62","-242,021.36","-241,208.87","49,939.36","16,520,788.93"
2,2024-01-04,8075,"19,897,396.67",0.00,784.80,622.30,462.80,"39,794.79",556.69,"309,477.48",...,"-432,603.62",0.00,0.00,"122,143.88","19,543.02","-141,686.90","-574,290.52","-540,937.12","56,272.51","18,948,483.86"
3,2024-01-05,8201,"20,169,052.44",0.00,773.20,639.80,471.90,"40,338.10",745.85,"313,609.98",...,"-77,597.03",0.00,0.00,"111,358.17","17,817.30","-129,175.47","-206,772.50","-197,304.48","57,064.13","19,558,033.00"
4,2024-01-08,24735,"55,993,206.29",0.00,"2,260.00","1,653.40","1,151.80","111,986.41","2,217.63","933,346.68",...,"-83,767.93",0.00,0.00,"288,787.71","46,206.04","-334,993.75","-418,761.68","-394,066.21","168,440.56","54,377,946.00"


In [135]:
deposit_date_cruce_final_desg.shape

(22, 39)

### CRUCE FINAL

In [136]:
deposit_date_cruce_4 = grrcn_summ_range.merge(deposit_date_cruce_final_desg[['fecha_pago','total_monto_comisiones',
                                                                     'total_monto_sobrerango',
                                                                    'Monto_ajustes_calc',
                                                                    'Monto_impuesto_calc',
                                                                    'Monto_neto_calc']],
                                                  on='fecha_pago',how='left')
deposit_date_cruce_4.head()

,1,2,3,4,fecha_pago,6,7,8,9,10,...,Monto_descuento,Monto_servicio,Monto_ajuste,Monto_impuestos,Monto_saldo_debito_inicial,total_monto_comisiones,total_monto_sobrerango,Monto_ajustes_calc,Monto_impuesto_calc,Monto_neto_calc
0,SUMMARY,7464284207,002,NaN,2024-01-02,MXN,NaN,"-995,203.00",0.00,"238,780.00",...,"2,387.80",24.00,0.00,385.88,"-7,154.35","1,433,075.10","8,031.00","-643,180.61","230,576.98","74,016,968.18"
1,SUMMARY,8138352885,002,3630036365,2024-01-02,MXN,NaN,"7,362,518,997.00","7,610,454,795.00","156,575,445.00",...,"1,565,754.45","7,984.80","-653,820.70","251,798.03",0.00,"1,433,075.10","8,031.00","-643,180.61","230,576.98","74,016,968.18"
2,SUMMARY,7464284207,002,NaN,2024-01-03,MXN,NaN,"-1,050,179.00",0.00,"47,779.00",...,477.79,8.50,14.33,77.80,"-9,952.03","310,463.39","1,657.60","-241,208.87","49,939.36","16,520,788.93"
3,SUMMARY,8138352885,002,0020024044,2024-01-03,MXN,NaN,"1,644,093,684.00","1,708,025,318.00","34,084,092.00",...,"340,840.92","1,642.50","-242,035.69","54,797.23",0.00,"310,463.39","1,657.60","-241,208.87","49,939.36","16,520,788.93"
4,SUMMARY,7464284207,002,NaN,2024-01-04,MXN,NaN,"-1,114,130.00",0.00,"54,390.00",...,543.90,7.40,0.00,88.21,"-10,501.79","349,833.29","1,869.90","-540,937.12","56,272.51","18,948,483.86"


In [137]:
deposit_date_cruce_final = deposit_date_cruce_4[['fecha_pago','Monto_bruto','Monto_descuento','Monto_servicio','Monto_ajuste',
                                                 'Monto_impuestos','Monto_saldo_debito_inicial','Monto_neto',
                                                 'total_monto_comisiones','total_monto_sobrerango','Monto_ajustes_calc',
                                                 'Monto_impuesto_calc','Monto_neto_calc']]
deposit_date_cruce_final.head()

,fecha_pago,Monto_bruto,Monto_descuento,Monto_servicio,Monto_ajuste,Monto_impuestos,Monto_saldo_debito_inicial,Monto_neto,total_monto_comisiones,total_monto_sobrerango,Monto_ajustes_calc,Monto_impuesto_calc,Monto_neto_calc
0,2024-01-02,0.00,"2,387.80",24.00,0.00,385.88,"-7,154.35","-9,952.03","1,433,075.10","8,031.00","-643,180.61","230,576.98","74,016,968.18"
1,2024-01-02,"76,104,547.95","1,565,754.45","7,984.80","-653,820.70","251,798.03",0.00,"73,625,189.97","1,433,075.10","8,031.00","-643,180.61","230,576.98","74,016,968.18"
2,2024-01-03,0.00,477.79,8.50,14.33,77.80,"-9,952.03","-10,501.79","310,463.39","1,657.60","-241,208.87","49,939.36","16,520,788.93"
3,2024-01-03,"17,080,253.18","340,840.92","1,642.50","-242,035.69","54,797.23",0.00,"16,440,936.84","310,463.39","1,657.60","-241,208.87","49,939.36","16,520,788.93"
4,2024-01-04,0.00,543.90,7.40,0.00,88.21,"-10,501.79","-11,141.30","349,833.29","1,869.90","-540,937.12","56,272.51","18,948,483.86"


In [138]:
deposit_date_cruce_final.shape

(44, 13)

### TRANSACCIONES POSITIVAS

In [139]:
deposit_success_txn = txn_grrcn_deposit[(txn_grrcn_deposit['Monto_transaccion'] >= 0)]

In [140]:
deposit_success_cruce_1 = deposit_success_txn.groupby(['fecha_pago']).agg({'transaction_id':'count',
                                                                      'Monto_transaccion':sum,
                                                                      'Monto_primera_cuota':sum,
                                                                      'Monto_cuota_siguiente':sum,
                                                                      'Monto_servicio':sum,
                                                                      'Monto_aceleracion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_success_cruce_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1416712927.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_success_cruce_1 = deposit_success_txn.groupby(['fecha_pago']).agg({'transaction_id':'count',


,fecha_pago,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,...,iva_monto_comision_network_fee,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa
0,2024-01-02,34450,"76,384,771.46","840,477.89","840,467.20",0.00,"-245,859.55",0.00,"3,534.40","2,792.30",...,"24,443.13",468.55,"204,511.73",0.00,"8,038.80","1,433,896.26","236,687.04","1,286.21","229,423.40","37,869.93"
1,2024-01-03,7213,"17,158,938.15","276,109.11","276,106.72",0.00,"-84,205.12",0.00,702.00,548.80,...,"5,490.86",62.32,"44,200.66",0.00,"1,661.60","310,961.47","83,492.35",265.86,"49,753.84","13,358.78"
2,2024-01-04,8072,"19,899,556.67","278,762.83","278,760.04",0.00,"-122,143.88",0.00,784.80,622.30,...,"6,367.86",89.07,"49,521.93",0.00,"1,869.90","349,867.85","93,390.95",299.18,"55,978.86","14,942.55"
3,2024-01-05,8197,"20,205,052.44","292,423.89","292,420.98",0.00,"-111,358.17",0.00,773.20,641.20,...,"6,465.62",119.34,"50,260.16",0.00,"1,886.30","355,281.93","103,196.08",301.81,"56,845.11","16,511.37"
4,2024-01-08,24725,"56,062,005.29","690,159.80","690,150.68",0.00,"-288,787.71",0.00,"2,261.20","1,654.10",...,"17,939.84",354.82,"149,533.75",0.00,"5,072.30","1,048,927.55","267,498.52",811.57,"167,828.41","42,799.76"


In [141]:
deposit_success_cruce_1.shape

(22, 29)

### CANCELACIONES

In [142]:
deposit_refunded_txn = txn_grrcn_deposit[(txn_grrcn_deposit['Monto_transaccion'] < 0)]

In [143]:
deposit_refunded_cruce_1 = deposit_refunded_txn.groupby(['fecha_pago']).agg({'transaction_id':'count',
                                                                      'Monto_transaccion':sum,
                                                                      'Monto_primera_cuota':sum,
                                                                      'Monto_cuota_siguiente':sum,
                                                                      'Monto_servicio':sum,
                                                                      'Monto_aceleracion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_refunded_cruce_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3553432280.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_refunded_cruce_1 = deposit_refunded_txn.groupby(['fecha_pago']).agg({'transaction_id':'count',


,fecha_pago,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,...,iva_monto_comision_network_fee,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa
0,2024-01-02,14,"-52,939.59",-177.12,-176.89,0.00,0.00,0.00,-0.80,-7.00,...,-16.94,0.00,-131.38,16.94,-7.80,-821.15,0.00,-1.25,-131.38,0.00
1,2024-01-03,4,"-34,880.00",0.00,0.00,0.00,0.00,0.00,0.00,-1.40,...,-11.16,0.00,-79.69,11.16,-4.00,-498.08,0.00,-0.64,-79.69,0.00
2,2024-01-04,3,"-2,160.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.69,0.00,-5.53,0.69,0.00,-34.56,0.00,0.00,-5.53,0.00
3,2024-01-05,4,"-36,000.00",0.00,0.00,0.00,0.00,0.00,0.00,-1.40,...,-11.52,0.00,-82.56,11.52,-1.40,-516.00,0.00,-0.22,-82.56,0.00
4,2024-01-08,10,"-68,799.00","-6,666.68","-6,666.66",0.00,0.00,0.00,-1.20,-0.70,...,-22.02,0.00,-198.28,22.02,-7.10,"-1,239.24",0.00,-1.14,-198.28,0.00


In [144]:
deposit_refunded_cruce_1.shape

(20, 29)

### POR FECHA DE DEPOSITO Y AFILIACIÓN 

In [145]:
deposit_afi_cruce_1 = txn_grrcn_deposit.groupby(['fecha_pago','numero_establecimiento','fecha_transaccion']).agg({
                                                                      'transaction_id':'count',                                                                  
                                                                      'Monto_transaccion':sum,
                                                                      'Monto_primera_cuota':sum,
                                                                      'Monto_cuota_siguiente':sum,
                                                                      'Monto_servicio':sum,
                                                                      'Monto_aceleracion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_afi_cruce_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\180223420.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_afi_cruce_1 = txn_grrcn_deposit.groupby(['fecha_pago','numero_establecimiento','fecha_transaccion']).agg({


,fecha_pago,numero_establecimiento,fecha_transaccion,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,...,iva_monto_comision_network_fee,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa
0,2024-01-02,8138464110,2023-12-27,2,"-6,720.00",0.00,0.00,0.00,0.00,0.00,...,-2.15,0.00,-17.20,2.15,-1.40,-107.52,0.00,-0.22,-17.20,0.00
1,2024-01-02,8138464110,2023-12-28,2,"-4,000.00",0.00,0.00,0.00,0.00,0.00,...,-1.28,0.00,-10.24,1.28,-0.80,-64.00,0.00,-0.13,-10.24,0.00
2,2024-01-02,8138464110,2023-12-29,175,"1,214,915.70","3,016.74","3,016.66",0.00,"-3,457.10",0.00,...,388.77,17.61,"3,110.18",0.00,96.90,"21,978.55","3,439.00",15.50,"3,516.57",550.24
3,2024-01-02,8138464110,2023-12-30,202,"1,176,499.13","6,333.35","6,333.33",0.00,"-1,919.00",0.00,...,376.48,21.31,"3,011.84",0.00,88.40,"21,310.18","1,900.00",14.14,"3,409.63",304.00
4,2024-01-02,8138464110,2023-12-31,175,"1,166,564.43","1,630.03","1,629.99",0.00,-388.19,0.00,...,373.30,0.00,"2,986.40",7.32,87.70,"21,043.92",384.00,14.03,"3,367.03",61.44


In [146]:
deposit_afi_cruce_1.shape

(641, 31)

# TRANSACTIONS GRRNC - CLIP

In [147]:
txn_grrcn_txn = cruce_1[(cruce_1['fecha_transaccion'] >= start_date) & (cruce_1['fecha_transaccion'] <= final_date)]
txn_grrcn_txn.sort_values(by=['fecha_transaccion','hora_transaccion'],inplace=True)
txn_grrcn_txn.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\860916731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_grrcn_txn.sort_values(by=['fecha_transaccion','hora_transaccion'],inplace=True)


,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
28737,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,2023-12-31,813846413640101,...,940.00,1,0.00000,376673,3011,8138464136,5812,OPTBLUE,940.00,True
28738,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464227,2023-12-29,2023-12-31,813846422740101,...,129.00,1,0.00000,371777,7005,8138464227,5921,OPTBLUE,129.00,True
28739,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,2023-12-31,813846413640101,...,240.00,1,0.00000,371773,8002,8138464136,5814,OPTBLUE,240.00,True
28740,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,2023-12-31,813846413640101,...,456.50,1,0.00000,376774,1017,8138464136,5813,OPTBLUE,456.50,True
28741,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,2023-12-31,813846413640101,...,800.00,1,0.00000,376669,1008,8138464136,5812,OPTBLUE,800.00,True


In [148]:
txn_grrcn_txn.shape

(385030, 102)

In [149]:
duplicates_2 = txn_grrcn_txn[txn_grrcn_txn.duplicated(['order_id'] , keep=False)]
duplicates_2.sort_values(by=['order_id','fecha_transaccion','hora_transaccion'],inplace=True)
duplicates_2.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1998658264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates_2.sort_values(by=['order_id','fecha_transaccion','hora_transaccion'],inplace=True)


,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
295211,TRANSACTN,7464284207,002,NaN,2024-01-26,MXN,8138464136,2024-01-25,2024-01-25,813846413640125,...,418.00,1,0.00000,341161,1017,8138464136,5814,OPTBLUE,418.00,True
295212,TRANSACTN,8138352885,002,0250054405,2024-01-26,MXN,8138464136,2024-01-25,2024-01-25,813846413640125,...,418.00,1,0.00000,341161,1017,8138464136,5814,OPTBLUE,418.00,True
162922,TRANSACTN,7464284207,002,NaN,2024-01-18,MXN,8138464136,2024-01-17,2024-01-17,813846413640117,...,40.00,1,0.00000,371777,2002,8138464136,5812,OPTBLUE,40.00,True
162923,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464136,2024-01-17,2024-01-17,813846413640117,...,40.00,1,0.00000,371777,2002,8138464136,5812,OPTBLUE,40.00,True
316991,TRANSACTN,7464284207,002,NaN,2024-01-29,MXN,8138464136,2024-01-26,2024-01-26,813846413640126,...,28.00,1,0.00000,371775,1020,8138464136,5812,OPTBLUE,28.00,True


In [150]:
duplicates_2.shape

(252768, 102)

In [151]:
txn_grrcn_txn.drop_duplicates(subset='order_id',keep='last',inplace=True)
txn_grrcn_txn.shape

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\1030073543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_grrcn_txn.drop_duplicates(subset='order_id',keep='last',inplace=True)


(258542, 102)

In [152]:
comparison_2 = txn_grrcn_txn[(txn_grrcn_txn['comparison'] == False)]
comparison_2.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
28765,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464193,2023-12-29,2023-12-31,813846419340101,...,"1,563.00",1,3.00000,376699,4004,8138464193,8299,OPTBLUE,"1,563.00",False
28938,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464136,2023-12-29,2023-12-31,813846413640101,...,543.00,1,3.00000,379911,7529,8138464136,5812,OPTBLUE,543.00,False
28951,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464227,2023-12-29,2023-12-31,813846422740101,...,"1,650.00",1,3.00000,371775,1009,8138464227,5045,OPTBLUE,"1,650.00",False
29010,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464144,2023-12-29,2024-01-01,813846414440101,...,420.00,1,6.00000,370787,2041,8138464144,7399,OPTBLUE,420.00,False
29040,TRANSACTN,8138352885,002,3630036365,2024-01-02,MXN,8138464227,2023-12-29,2024-01-01,813846422740101,...,"1,530.00",1,3.00000,371778,1000,8138464227,5651,OPTBLUE,"1,530.00",False


In [153]:
comparison_2.shape

(1228, 102)

# GROUP BY TRANSACTION DATE

In [154]:
txn_txn_date_cruce_1 = txn_grrcn_txn.groupby(['fecha_transaccion']).agg({
    'transaction_id':'count',
    'Monto_transaccion':sum,
    'Monto_primera_cuota':sum,
    'Monto_cuota_siguiente':sum,
    'Monto_servicio':sum,
    'Monto_aceleracion':sum,
    'monto_sobrerango_v0':sum,
    'monto_sobrerango_v1':sum,
    'monto_sobrerango_v2':sum,
    'monto_sobrerango_v3':sum,
    'monto_comision_network_fee':sum,
    'monto_comision_inbound_fee':sum,
    'monto_comision_industria_fee':sum,
    'monto_comision_cancelacion':sum,
    'iva_monto_sobrerango_v0':sum,
    'iva_monto_sobrerango_v1':sum,
    'iva_monto_sobrerango_v2':sum,
    'iva_monto_sobrerango_v3':sum,
    'iva_monto_comision_network_fee':sum,
    'iva_monto_comision_inbound_fee':sum,
    'iva_monto_comision_industria_fee':sum,
    'iva_monto_comision_cancelacion':sum,
    'total_monto_sobrerango':sum,
    'total_monto_comisiones':sum,
    'monto_sobretasa':sum,
    'total_iva_monto_sobrerango':sum,
    'total_iva_monto_comisiones':sum,
    'iva_monto_sobretasa':sum,
    'amount_new':sum,
    'order_id':'count'}).reset_index()
txn_txn_date_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_date_cruce_1['diff'] = txn_txn_date_cruce_1['Monto_transaccion'] - txn_txn_date_cruce_1['amount_new']
txn_txn_date_cruce_1.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\3416881863.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_date_cruce_1 = txn_grrcn_txn.groupby(['fecha_transaccion']).agg({


,fecha_transaccion,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,...,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,amount_new,number_transactions,diff
0,2024-01-01,5087,"11,996,619.66","101,169.58","101,168.60",0.00,"-27,680.92",0.00,520.40,513.80,...,2.56,"1,313.70","235,092.83","27,431.54",210.19,"37,614.85","4,389.05","11,992,334.67",5087,"4,284.99"
1,2024-01-02,6740,"15,904,078.66","228,115.21","228,113.13",0.00,"-58,493.24",0.00,661.20,519.40,...,4.59,"1,569.30","290,998.17","57,948.81",251.09,"46,559.71","9,271.81","15,940,284.32",6740,"-36,205.66"
2,2024-01-03,7910,"19,993,151.82","285,450.61","285,447.97",0.00,"-136,446.13",0.00,759.60,609.00,...,0.64,"1,830.10","350,956.74","107,623.67",292.82,"56,153.08","17,219.79","19,950,565.10",7910,"42,586.72"
3,2024-01-04,8187,"19,786,615.18","265,890.10","265,887.18",0.00,"-95,437.20",0.00,762.00,664.30,...,1.60,"1,889.10","352,273.01","87,371.95",302.26,"56,363.68","13,979.51","19,804,713.41",8187,"-18,098.23"
4,2024-01-05,8580,"23,736,473.05","313,535.87","313,532.59",0.00,"-118,443.39",0.00,796.00,620.90,...,12.35,"1,949.90","430,319.33","117,535.52",311.98,"68,851.09","18,805.68","23,721,353.92",8580,"15,119.13"


In [155]:
txn_txn_date_cruce_1.shape

(31, 32)

# GROUP BY TRANSACTION DATE AND STATUS CODE

In [156]:
txn_txn_status_cruce_1 = txn_grrcn_txn.groupby(['fecha_transaccion','status_code','numero_establecimiento']).agg({
    'transaction_id':'count',
    'Monto_transaccion':sum,
    'Monto_primera_cuota':sum,
    'Monto_cuota_siguiente':sum,
    'Monto_servicio':sum,
    'Monto_aceleracion':sum,
    'monto_sobrerango_v0':sum,
    'monto_sobrerango_v1':sum,
    'monto_sobrerango_v2':sum,
    'monto_sobrerango_v3':sum,
    'monto_comision_network_fee':sum,
    'monto_comision_inbound_fee':sum,
    'monto_comision_industria_fee':sum,
    'monto_comision_cancelacion':sum,
    'iva_monto_sobrerango_v0':sum,
    'iva_monto_sobrerango_v1':sum,
    'iva_monto_sobrerango_v2':sum,
    'iva_monto_sobrerango_v3':sum,
    'iva_monto_comision_network_fee':sum,
    'iva_monto_comision_inbound_fee':sum,
    'iva_monto_comision_industria_fee':sum,
    'iva_monto_comision_cancelacion':sum,
    'total_monto_sobrerango':sum,
    'total_monto_comisiones':sum,
    'monto_sobretasa':sum,
    'total_iva_monto_sobrerango':sum,
    'total_iva_monto_comisiones':sum,
    'iva_monto_sobretasa':sum,
    'amount_new':sum,
    'order_id':'count'}).reset_index()
txn_txn_status_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_status_cruce_1['diff'] = txn_txn_status_cruce_1['Monto_transaccion'] - txn_txn_status_cruce_1['amount_new']
txn_txn_status_cruce_1.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\600782973.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_status_cruce_1 = txn_grrcn_txn.groupby(['fecha_transaccion','status_code','numero_establecimiento']).agg({


,fecha_transaccion,status_code,numero_establecimiento,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,...,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,amount_new,number_transactions,diff
0,2024-01-01,1,8138464110,103,"507,970.75","3,813.34","3,813.33",0.00,-291.72,0.00,...,0.00,44.90,"9,172.93",286.00,7.18,"1,467.67",45.76,"507,970.75",103,0.00
1,2024-01-01,1,8138464128,133,"320,044.15",0.00,0.00,0.00,0.00,0.00,...,0.00,42.40,"4,823.46",0.00,6.78,771.75,0.00,"320,044.15",133,0.00
2,2024-01-01,1,8138464136,3050,"6,190,987.23","13,102.43","13,102.14",0.00,"-3,269.65",0.00,...,0.00,738.50,"133,313.61","3,237.22",118.16,"21,330.18",517.95,"6,188,020.91",3050,"2,966.32"
3,2024-01-01,1,8138464144,89,"470,581.83","21,970.07","21,970.06",0.00,"-8,069.75",0.00,...,0.00,35.00,"9,646.93","8,002.93",5.60,"1,543.51","1,280.47","469,355.96",89,"1,225.87"
4,2024-01-01,1,8138464151,13,"10,784.94",0.00,0.00,0.00,0.00,0.00,...,0.00,2.00,134.81,0.00,0.32,21.57,0.00,"10,784.94",13,0.00


In [157]:
txn_txn_status_cruce_1.dtypes

fecha_transaccion                   datetime64[ns]
status_code                                 object
numero_establecimiento                      object
transaction_id                               int64
Monto_transaccion                          float64
Monto_primera_cuota                        float64
Monto_cuota_siguiente                      float64
Monto_servicio                             float64
Monto_aceleracion                          float64
monto_sobrerango_v0                        float64
monto_sobrerango_v1                        float64
monto_sobrerango_v2                        float64
monto_sobrerango_v3                        float64
monto_comision_network_fee                 float64
monto_comision_inbound_fee                 float64
monto_comision_industria_fee               float64
monto_comision_cancelacion                 float64
iva_monto_sobrerango_v0                    float64
iva_monto_sobrerango_v1                    float64
iva_monto_sobrerango_v2        

In [158]:
txn_txn_status_cruce_1.shape

(410, 34)

### STATUS 1

In [159]:
txn_txn_status_cruce_1_status_1 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '1')]
txn_txn_status_cruce_1_status_1.head()

,fecha_transaccion,status_code,numero_establecimiento,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,...,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,amount_new,number_transactions,diff
0,2024-01-01,1,8138464110,103,"507,970.75","3,813.34","3,813.33",0.00,-291.72,0.00,...,0.00,44.90,"9,172.93",286.00,7.18,"1,467.67",45.76,"507,970.75",103,0.00
1,2024-01-01,1,8138464128,133,"320,044.15",0.00,0.00,0.00,0.00,0.00,...,0.00,42.40,"4,823.46",0.00,6.78,771.75,0.00,"320,044.15",133,0.00
2,2024-01-01,1,8138464136,3050,"6,190,987.23","13,102.43","13,102.14",0.00,"-3,269.65",0.00,...,0.00,738.50,"133,313.61","3,237.22",118.16,"21,330.18",517.95,"6,188,020.91",3050,"2,966.32"
3,2024-01-01,1,8138464144,89,"470,581.83","21,970.07","21,970.06",0.00,"-8,069.75",0.00,...,0.00,35.00,"9,646.93","8,002.93",5.60,"1,543.51","1,280.47","469,355.96",89,"1,225.87"
4,2024-01-01,1,8138464151,13,"10,784.94",0.00,0.00,0.00,0.00,0.00,...,0.00,2.00,134.81,0.00,0.32,21.57,0.00,"10,784.94",13,0.00


In [160]:
txn_txn_status_cruce_1_status_1.shape

(341, 34)

### STATUS 3

In [161]:
txn_txn_status_cruce_1_status_3 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '3')]
txn_txn_status_cruce_1_status_3.head()

,fecha_transaccion,status_code,numero_establecimiento,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,...,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,amount_new,number_transactions,diff
12,2024-01-01,3,8138464441,1,"5,000.00",0.00,0.00,0.00,0.00,0.00,...,0.00,0.70,30.00,0.00,0.11,4.80,0.00,"-5,000.00",1,"10,000.00"
64,2024-01-05,3,8138464110,1,"-20,000.00","-6,666.68","-6,666.66",0.00,0.00,0.00,...,6.40,-1.30,-320.00,0.00,-0.21,-51.20,0.00,"-20,000.00",1,0.00
65,2024-01-05,3,8138464136,2,"-16,502.00",0.00,0.00,0.00,0.00,0.00,...,5.28,-1.30,-321.79,0.00,-0.21,-51.49,0.00,"-16,502.00",2,0.00
66,2024-01-05,3,8138464177,1,"-2,097.00",0.00,0.00,0.00,0.00,0.00,...,0.67,-0.40,-37.75,0.00,-0.06,-6.04,0.00,"-2,097.00",1,0.00
80,2024-01-06,3,8138464136,1,-414.00,0.00,0.00,0.00,0.00,0.00,...,0.13,0.00,-8.07,0.00,0.00,-1.29,0.00,-414.00,1,0.00


In [162]:
txn_txn_status_cruce_1_status_3.shape

(25, 34)

# POSIBLES DISPUTAS

In [163]:
posibles_disputas = txn_grrcn_txn[(txn_grrcn_txn['status_code'] != '1') & (txn_grrcn_txn['status_code'] != '3')]
posibles_disputas.head()


,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,amount,status_code,msi,bin,last4,affiliation,mcc,affiliation_type,amount_new,comparison
30139,TRANSACTN,8138352885,002,0040044357,2024-01-05,MXN,8138464110,2024-01-04,2024-01-04,813846411040104,...,"8,000.00",2,0.00000,377661,0778,8138464110,4722,OPTBLUE,"8,000.00",False
35095,TRANSACTN,8138352885,002,0190096084,2024-01-22,MXN,8138464110,2024-01-19,2024-01-19,813846411040119,...,"10,000.00",2,0.00000,379796,1014,8138464110,4722,OPTBLUE,"10,000.00",False
35436,TRANSACTN,8138352885,002,0120026148,2024-01-15,MXN,8138464110,2024-01-12,2024-01-13,813846411040113,...,"1,855.00",2,0.00000,373916,1004,8138464110,4722,OPTBLUE,"1,855.00",False
40102,TRANSACTN,8138352885,002,0170074943,2024-01-18,MXN,8138464144,2024-01-17,2024-01-17,813846414440118,...,"2,500.00",2,0.00000,376669,3026,8138464144,7399,OPTBLUE,"2,500.00",False
43625,TRANSACTN,8138352885,002,0180084340,2024-01-19,MXN,8138464441,2024-01-18,2024-01-18,813846444140118,...,"2,000.00",2,0.00000,379003,4003,8138464441,8021,OPTBLUE,"2,000.00",False


In [164]:
posibles_disputas.shape

(52, 102)

In [165]:
txn_txn_status_cruce_1_posibles_disputas = posibles_disputas.groupby(['fecha_transaccion','numero_establecimiento']).agg({
    'transaction_id':'count',
    'Monto_transaccion':sum,
    'Monto_primera_cuota':sum,
    'Monto_cuota_siguiente':sum,
    'Monto_servicio':sum,
    'Monto_aceleracion':sum,
    'monto_sobrerango_v0':sum,
    'monto_sobrerango_v1':sum,
    'monto_sobrerango_v2':sum,
    'monto_sobrerango_v3':sum,
    'monto_comision_network_fee':sum,
    'monto_comision_inbound_fee':sum,
    'monto_comision_industria_fee':sum,
    'monto_comision_cancelacion':sum,
    'iva_monto_sobrerango_v0':sum,
    'iva_monto_sobrerango_v1':sum,
    'iva_monto_sobrerango_v2':sum,
    'iva_monto_sobrerango_v3':sum,
    'iva_monto_comision_network_fee':sum,
    'iva_monto_comision_inbound_fee':sum,
    'iva_monto_comision_industria_fee':sum,
    'iva_monto_comision_cancelacion':sum,
    'total_monto_sobrerango':sum,
    'total_monto_comisiones':sum,
    'monto_sobretasa':sum,
    'total_iva_monto_sobrerango':sum,
    'total_iva_monto_comisiones':sum,
    'iva_monto_sobretasa':sum,
    'amount_new':sum,
    'order_id':'count'}).reset_index()
txn_txn_status_cruce_1_posibles_disputas.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_23168\2852950514.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_status_cruce_1_posibles_disputas = posibles_disputas.groupby(['fecha_transaccion','numero_establecimiento']).agg({


,fecha_transaccion,numero_establecimiento,transaction_id,Monto_transaccion,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,monto_sobrerango_v0,monto_sobrerango_v1,...,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,amount_new,order_id
0,2024-01-01,8138464110,1,"-8,000.00",0.00,0.00,0.00,0.00,0.00,0.00,...,-20.48,2.56,-0.70,-128.00,0.00,-0.11,-20.48,0.00,"8,000.00",1
1,2024-01-02,8138464110,2,"-11,855.00",0.00,0.00,0.00,0.00,0.00,-0.40,...,-30.35,3.79,-0.40,-189.68,0.00,-0.06,-30.35,0.00,"11,855.00",2
2,2024-01-02,8138464144,1,"-2,500.00",0.00,0.00,0.00,0.00,0.00,-0.40,...,-7.40,0.80,-0.40,-46.25,0.00,-0.06,-7.40,0.00,"2,500.00",1
3,2024-01-03,8138464441,1,"-2,000.00",0.00,0.00,0.00,0.00,0.00,-0.40,...,-1.28,0.64,-0.40,-8.00,0.00,-0.06,-1.28,0.00,"2,000.00",1
4,2024-01-04,8138464110,1,"-5,000.00",0.00,0.00,0.00,0.00,0.00,0.00,...,-12.80,1.60,-0.70,-80.00,0.00,-0.11,-12.80,0.00,"5,000.00",1


In [166]:
txn_txn_status_cruce_1_posibles_disputas.shape

(43, 32)

# EXCEL GRRCN 

In [167]:
writer = pd.ExcelWriter('GRRCN ENERO 2024.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
# grrcn_file.to_excel(writer, sheet_name='GRRCN ALL',index=False)
grrcn_head.to_excel(writer, sheet_name='GRRCN HEADER',index=False)
grrcn_summ_range.to_excel(writer, sheet_name='GRRCN SUMMARY',index=False)
grrcn_subm_range.to_excel(writer, sheet_name='GRRCN SUBMISSION',index=False)
grrcn_tran_range.to_excel(writer, sheet_name='GRRCN TRANSACTION',index=False)
grrcn_txnp_range_1.to_excel(writer, sheet_name='GRRCN PRICING_1',index=False)
grrcn_txnp_range_2.to_excel(writer, sheet_name='GRRCN PRICING_2',index=False)
grrcn_txnp_range_3.to_excel(writer, sheet_name='GRRCN PRICING_3',index=False)
grrcn_txnp_range_4.to_excel(writer, sheet_name='GRRCN PRICING_4',index=False)
grrcn_char_range.to_excel(writer, sheet_name='GRRCN CHARGEBACKS',index=False)
grrcn_adju_range.to_excel(writer, sheet_name='GRRCN ADJUSTMENTS',index=False)
grrcn_trai.to_excel(writer, sheet_name='GRRCN TRAILER',index=False)
writer.close()

# PAYOUTS GRRNC

In [ ]:
writer = pd.ExcelWriter('OPTBLUE PAYOUTS ENERO 2024.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
deposit_date_cruce_final.to_excel(writer, sheet_name='PAYOUTS',index=False)
deposit_date_cruce_final_desg.to_excel(writer, sheet_name='PAYOUTS DESGLOSADO',index=False)
deposit_date_msi.to_excel(writer, sheet_name='PAYOUTS POR MSI',index=False)
deposit_success_cruce_1.to_excel(writer, sheet_name='PAYOUTS SUCCESS TXN',index=False)
deposit_refunded_cruce_1.to_excel(writer, sheet_name='PAYOUTS REFUNDED TXN',index=False)
deposit_afi_cruce_1.to_excel(writer, sheet_name='PAYOUTS AFILIACION',index=False)
txn_grrcn_deposit.to_excel(writer, sheet_name='PAYOUTS DETAIL',index=False)
cruce_1_rej.to_excel(writer, sheet_name='RECHAZADOS AMEX',index=False)
duplicates_1.to_excel(writer, sheet_name='PAYOUTS DUPLICATES',index=False)
comparison_1.to_excel(writer, sheet_name='PAYOUTS COMPARISON',index=False)
not_found_txn_epa_deposit.to_excel(writer, sheet_name='NO ENCONTRADOS',index=False)
grrcn_char.to_excel(writer, sheet_name='CHARGEBACKS DETAIL',index=False)
deposit_date_chbk.to_excel(writer, sheet_name='CHARGEBACKS BY DATE',index=False)
deposit_date_chnk_positivos.to_excel(writer, sheet_name='CHARGEBACKS POSITIVOS',index=False)
deposit_date_chnk_negativos.to_excel(writer, sheet_name='CHARGEBACKS NEGATIVOS',index=False)
grrcn_adju.to_excel(writer, sheet_name='ADJUSTMENTS DETAIL',index=False)
deposit_date_adj.to_excel(writer, sheet_name='ADJUSTMENTS BY DATE',index=False)
deposit_date_adj_positivos.to_excel(writer, sheet_name='ADJUSTMENTS POSITIVOS',index=False)
deposit_date_adj_negativos.to_excel(writer, sheet_name='ADJUSTMENTS NEGATIVOS',index=False)
writer.close()

# GRRCN TRANSACCIONES

In [ ]:
writer = pd.ExcelWriter('OPTBLUE TRANSACCIONES ENERO 2024.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
txn_grrcn_txn.to_excel(writer, sheet_name='TRANSACTIONS DETAIL',index=False)
duplicates_2.to_excel(writer, sheet_name='TRANSACTIONS DUPLICATES',index=False)
comparison_2.to_excel(writer, sheet_name='TRANSACTIONS COMPARISON',index=False)
txn_txn_date_cruce_1.to_excel(writer, sheet_name='TXN DATE',index=False)
txn_txn_status_cruce_1.to_excel(writer, sheet_name='TXN DATE AND STATUS',index=False)
txn_txn_status_cruce_1_status_1.to_excel(writer, sheet_name='STATUS 1',index=False)
txn_txn_status_cruce_1_status_3.to_excel(writer, sheet_name='STATUS 3',index=False)
txn_txn_status_cruce_1_posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS DATE-AFFI',index=False)
posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS',index=False)
writer.close()